In [ ]:
!pip install gensim pyarabic gdown kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 5.1 MB/s 


# Preprocessing

In [ ]:
# use pyarabic module for cleaning Arabic Words
from pyarabic.araby import strip_tashkeel, strip_diacritics, strip_tatweel, tokenize
from string import punctuation

def clean(text, model_name):
  '''
  Get the clean word from the given word, according to the following rules.
  1. Remove tashkeel/diacritics/punctuation/tatweel
  
  @param text: the text to clean
  @type text: str
  @return: the clean text
  @rtype: list
  '''

  arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
  english_punctuations = punctuation
  punctuations_list = (arabic_punctuations + english_punctuations).replace('_', '')
  
  text = strip_tashkeel(text)
  text = strip_diacritics(text)
  text = strip_tatweel(text)

  if model_name in ['TWITTER', 'WIKI']:
    # skip under_score
    search  = \
    ["أ", "إ", "آ", "ة", " ", "-", "/", ".", "،", " و ", " يا ", '"', "'", "ى", "\\", '\n' , '\t' , '&quot;',  '?',   '؟',   '!', '»', '«']
    replace = \
    ["ا", "ا", "ا", "ه", " ", " ",  "",  "",  "",  " و",  " يا",  "",  "", "ي",   "",       ' ',  ' ',  ' ', ' ? ', ' ؟ ', ' ! ',  '', '' ]
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    for i in range(0, len(search)):
      text = text.replace(search[i], replace[i])     
  
  if model_name != 'TWITTER':
    text = text.replace('_', '')  
  
  # clean punctuations
  for i in text:
    if(i in punctuations_list):
      text = text.replace(i,'')
  
  # tokenize
  text = tokenize(text)
  return text

In [ ]:
a = clean('{><*&%][بسم  -  ', 'TWITTER')
b = clean('{><*&%][بسم  -  ', 'FASTTEXT')
c = clean('[بسم الله - الرحمن * الرحيم]', 'WIKI')
d = clean("بِسمِ اللَّهِ الرَّحمـٰنِ الرَّحيمِ الحَمدُ لِلَّهِ رَبِّ العالَمينَ الرَّحمـٰنِ الرَّحيمِ مالِكِ يَومِ الدّينِ إِيّاكَ نَعبُدُ وَإِيّاكَ نَستَعينُ", 'TWITTER')
e = clean("بِسمِ اللَّهِ الرَّحمـٰنِ الرَّحيمِ الحَمدُ لِلَّهِ رَبِّ العالَمينَ الرَّحمـٰنِ الرَّحيمِ مالِكِ يَومِ الدّينِ إِيّاكَ نَعبُدُ وَإِيّاكَ نَستَعينُ ", 'KSUCCA')
for item in [a, b, c, d, e]:
  print(item)

['بسم']
['بسم']
['بسم', 'الله', 'الرحمن', 'الرحيم']
['بسم', 'الله', 'الرحمن', 'الرحيم', 'الحمد', 'لله', 'رب', 'العالمين', 'الرحمن', 'الرحيم', 'مالك', 'يوم', 'الدين', 'اياك', 'نعبد', 'واياك', 'نستعين']
['بسم', 'الله', 'الرحمن', 'الرحيم', 'الحمد', 'لله', 'رب', 'العالمين', 'الرحمن', 'الرحيم', 'مالك', 'يوم', 'الدين', 'إياك', 'نعبد', 'وإياك', 'نستعين']


In [ ]:
a = clean('بر_الوالدين', 'TWITTER')
b = clean('بر_الوالدين', 'WIKI')
c = clean('بر_الوالدين', 'KSUCCA')
d = clean('بر_الوالدين', 'FASTTEXT')
[a, b, c, d]

[['بر_الوالدين'], ['برالوالدين'], ['برالوالدين'], ['برالوالدين']]

# Word Embedding

## Download Models

### AraVec cbow_300_wiki


In [ ]:
# Download the pretrained model (AraVec cbow_300_wiki) via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_wiki.zip" 
!unzip "full_grams_cbow_300_wiki.zip"

--2022-07-28 15:43:06--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_wiki.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1491895880 (1.4G) [application/zip]
Saving to: ‘full_grams_cbow_300_wiki.zip’

full_grams_cbow_300 100%[===================>]   1.39G   121MB/s    in 11s     

2022-07-28 15:43:17 (126 MB/s) - ‘full_grams_cbow_300_wiki.zip’ saved [1491895880/1491895880]

Archive:  full_grams_cbow_300_wiki.zip
  inflating: full_grams_cbow_300_wiki.mdl  
  inflating: full_grams_cbow_300_wiki.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_300_wiki.mdl.wv.vectors.npy  


### AraVec cbow_300_twitter

In [ ]:
# Download the pretrained model (AraVec cbow_300_twitter) via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_twitter.zip" 
!unzip "full_grams_cbow_300_twitter.zip"

--2022-07-28 15:43:37--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_300_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3325529808 (3.1G) [application/zip]
Saving to: ‘full_grams_cbow_300_twitter.zip’

full_grams_cbow_300 100%[===================>]   3.10G   126MB/s    in 26s     

2022-07-28 15:44:03 (122 MB/s) - ‘full_grams_cbow_300_twitter.zip’ saved [3325529808/3325529808]

Archive:  full_grams_cbow_300_twitter.zip
  inflating: full_grams_cbow_300_twitter.mdl  
  inflating: full_grams_cbow_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_300_twitter.mdl.wv.vectors.npy  


### KSUCCA full_cbow

In [30]:
# Download the pretrained model (KSUCCA full_cbow) via terminal commands
!gdown https://drive.google.com/uc?id=1rZiOKy71Z_WycxnOG9bwrNoAc4ziGo_n

Downloading...
From: https://drive.google.com/uc?id=1rZiOKy71Z_WycxnOG9bwrNoAc4ziGo_n
To: /content/ksucca_full_cbow.bin
100% 453M/453M [00:05<00:00, 76.0MB/s]


### FastText Arabic CBOW-300

In [ ]:
# Download the pretrained model (FastText Arabic version - CBOW300) via terminal commands
!gdown https://drive.google.com/uc?id=1MHtDd5gRvqWGQvmX76wcjvmjaN_gHjst
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d javadhelali/fasttext-pretrained-arabic-word-vectors
!unzip fasttext-pretrained-arabic-word-vectors.zip

Downloading...
From: https://drive.google.com/uc?id=1MHtDd5gRvqWGQvmX76wcjvmjaN_gHjst
To: /content/kaggle.json
100% 69.0/69.0 [00:00<00:00, 67.6kB/s]
 99% 1.19G/1.20G [00:08<00:00, 131MB/s]
100% 1.20G/1.20G [00:08<00:00, 153MB/s]
Archive:  fasttext-pretrained-arabic-word-vectors.zip
  inflating: cc.ar.300.vec           


## Load the pre-trained word2vec models

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# load the wiki model
model_wiki = Word2Vec.load("full_grams_cbow_300_wiki.mdl").wv
print("We've", len(model_wiki.index2word), "vocabularies")

We've 662109 vocabularies


In [ ]:
# load the tweets model
model_tw = Word2Vec.load("full_grams_cbow_300_twitter.mdl").wv
print("We've", len(model_tw.index2word), "vocabularies")

We've 1476715 vocabularies


In [ ]:
# load the ksucca model
model_ksucca = KeyedVectors.load_word2vec_format('ksucca_full_cbow.bin', binary=True)
print("We've", len(model_ksucca.index2word), "vocabularies")

We've 373108 vocabularies


In [ ]:
# load the facebook model
model_fasttext = KeyedVectors.load_word2vec_format("cc.ar.300.vec")
print("We've", len(model_fasttext.index2word), "vocabularies")

We've 2000000 vocabularies


### Testing Word Vectors

In [ ]:
model_tw[clean('طمأنينة', 'TWITTER')]

array([[-1.47282219e+00,  1.50031269e+00,  4.70157355e-01,
         1.37608397e+00, -1.45620489e+00, -8.02452862e-01,
         3.57282013e-01,  6.24089897e-01,  1.12618184e+00,
         2.64639425e+00,  7.68023610e-01, -9.11832511e-01,
         6.76315665e-01,  8.89694214e-01, -6.03546441e-01,
        -6.29395127e-01, -4.69495207e-02, -1.05934799e+00,
         1.92111397e+00, -3.00064659e+00,  2.72908688e+00,
         1.52707577e+00,  1.88424015e+00, -7.39471614e-01,
         1.34523273e+00, -1.16589285e-01,  5.55610895e-01,
         4.17886227e-02,  2.74039531e+00, -6.77424610e-01,
         6.40259802e-01, -5.86225271e-01, -8.67139995e-01,
         9.73269224e-01, -5.37413955e-02, -3.67704773e+00,
         1.32014918e+00,  5.60855746e-01, -1.01450217e+00,
        -1.13682508e+00, -1.21649897e+00, -1.85342121e+00,
         1.42324996e+00, -8.73523772e-01, -2.19200206e+00,
         1.34154880e+00, -4.06686723e-01, -7.60734916e-01,
        -4.75870043e-01,  5.79977691e-01, -2.83172578e-0

In [ ]:
model_wiki[clean('طمأنينة', 'WIKI')]

array([[-0.28953964, -0.07845741, -0.14822257, -0.24870284, -0.3346069 ,
        -0.35259336, -0.15048546,  0.17176072, -0.0734706 ,  0.05353372,
         0.11514466,  0.07408953,  0.02928444,  0.17113058,  0.3234126 ,
         0.05761059,  0.3559551 ,  0.00569709,  0.31998977, -0.09200501,
         0.12805058, -0.08255149, -0.01604828,  0.12156817, -0.09685886,
        -0.14863043, -0.2645954 , -0.15860076, -0.07299021,  0.2367228 ,
         0.11435907,  0.05433336, -0.42964724,  0.14958218, -0.45813102,
         0.11301208, -0.24789785, -0.11452873,  0.08234931,  0.15759626,
         0.20699564,  0.32070798,  0.10597168, -0.05420626,  0.07718668,
         0.02704845, -0.04396065, -0.07477729, -0.23346251,  0.05618991,
        -0.06177188,  0.09617444, -0.05075095, -0.31580532,  0.08684124,
         0.16714394,  0.00384073,  0.23903123, -0.06545994,  0.12655328,
         0.00561113, -0.23787743,  0.04055891, -0.09861883,  0.1357034 ,
        -0.03220984,  0.00254897,  0.09549627, -0.2

In [31]:
model_ksucca[clean('طمأنينة', 'KSUCCA')]

array([[ 8.61085415e-01, -3.71756762e-01,  3.10089529e-01,
        -2.31813788e-01,  1.87548175e-01,  2.53375024e-01,
         5.43829143e-01,  1.43743691e-03, -1.00014973e-02,
         3.61838996e-01,  1.65036210e-04, -8.24214816e-02,
        -4.34678167e-01, -6.12507947e-02, -4.64695960e-01,
         3.99640761e-02, -2.40052745e-01,  6.21533453e-01,
        -7.50877708e-02,  3.36336285e-01, -2.18026444e-01,
        -2.10178748e-01,  1.52601190e-02, -2.75114357e-01,
         2.05008358e-01,  4.28911209e-01, -2.26168543e-01,
         3.69684339e-01,  1.68559521e-01, -2.78831840e-01,
         9.88368019e-02, -2.85934955e-01,  1.82451978e-01,
        -8.72871697e-01, -1.78821851e-03, -1.38522699e-01,
        -2.66480654e-01,  1.33006647e-01,  2.83501357e-01,
        -4.75471407e-01,  1.37368441e-01, -3.46045554e-01,
        -3.93429577e-01, -3.51104885e-01,  1.69493631e-01,
        -1.39072940e-01,  3.73444632e-02,  6.17265105e-01,
         1.49523750e-01, -1.22152142e-01,  1.37822196e-0

In [ ]:
model_fasttext[clean('طمأنينة', 'FASTTEXT')]

array([[-3.800e-02,  4.730e-02, -4.800e-03,  2.210e-02,  8.730e-02,
        -2.080e-02, -8.640e-02,  2.680e-02, -4.870e-02, -1.940e-02,
        -1.460e-02,  2.420e-02, -5.440e-02,  8.220e-02, -1.100e-02,
         1.500e-03, -8.700e-03, -7.410e-02, -2.330e-02, -1.300e-03,
         4.230e-02,  3.070e-02, -1.190e-02, -4.290e-02,  6.800e-03,
         2.250e-02, -4.900e-02, -2.240e-02, -3.460e-02,  1.570e-02,
        -1.400e-02,  1.090e-02,  6.050e-02, -7.850e-02,  2.540e-02,
         8.790e-02,  1.150e-02, -4.190e-02,  2.270e-02,  2.070e-02,
        -2.630e-02,  2.860e-02,  6.880e-02, -2.200e-03,  1.420e-02,
         8.100e-02, -4.810e-02,  2.910e-02,  3.780e-02, -2.150e-02,
         1.130e-02,  2.200e-02,  7.380e-02,  2.310e-02, -4.110e-02,
        -3.890e-02,  8.700e-03,  4.470e-02, -7.730e-02,  1.500e-02,
         4.970e-02, -3.330e-02, -2.170e-02, -1.180e-02, -5.760e-02,
         6.680e-02, -1.660e-02, -7.600e-02, -7.490e-02,  9.270e-02,
        -3.480e-02,  3.300e-02,  1.920e-02,  1.9

In [ ]:
print("أهواء" in model_fasttext)
print("أهواء" in model_tw)
print("كلمه" in model_fasttext)

True
False
True


In [ ]:
a = model_fasttext.similarity('طمانينة', "طمأنينة")
b = model_fasttext.similarity('كلمة', "كلمه")
c = model_fasttext.similarity('كرة', 'كره')
d = model_fasttext.similarity('أحمد', 'احمد')
e = model_fasttext.similarity('إيمان', 'ايمان')
[a, b, c, d, e]

# fasttext knows the diff

[0.5381147, 0.730535, 0.49925467, 0.7931661, 0.7393669]

## The Holy Quran Text Cleaning

In [32]:
# download quran-simple and quran-simple-clean from tanzil
!gdown https://drive.google.com/uc?id=1lIX3raBe_TVmcJRJw8nX3S21Ab2qoaGB
!gdown https://drive.google.com/uc?id=1yWkmOg7isP7EnQtgsRuyEwrT5s28zxlr

Downloading...
From: https://drive.google.com/uc?id=1lIX3raBe_TVmcJRJw8nX3S21Ab2qoaGB
To: /content/quran-simple.txt
100% 1.32M/1.32M [00:00<00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yWkmOg7isP7EnQtgsRuyEwrT5s28zxlr
To: /content/quran-simple-clean.txt
100% 744k/744k [00:00<00:00, 94.9MB/s]


In [ ]:
# prepare/read Quran data
quran_clean_text = open("quran-simple-clean.txt").readlines()

# use this instead to display well-formed verses
quran_simple = open("quran-simple.txt").readlines()

print(quran_clean_text[:5])

quran_clean_text = [verse[:-1] for verse in quran_clean_text]
print(quran_clean_text[:5])

['بسم الله الرحمن الرحيم\n', 'الحمد لله رب العالمين\n', 'الرحمن الرحيم\n', 'مالك يوم الدين\n', 'إياك نعبد وإياك نستعين\n']
['بسم الله الرحمن الرحيم', 'الحمد لله رب العالمين', 'الرحمن الرحيم', 'مالك يوم الدين', 'إياك نعبد وإياك نستعين']


## Methods to get the most similar verses by a single word (Word Search)

### Max Similarity Score

Comparing vectors between the sinlge query word and each word in a verse, returning the max similarity of a word in a verse.

In [35]:
def get_verse_max_score(query_word, verse_text, model):
    '''
    Get the max similarity of query word and each word in a verse text.
    Example:
    >>> query_word = clean("الصلاة", 'KSUCCA')[0]
    >>> verse_text = clean("صلاة القيام", 'KSUCCA')
    >>> get_verse_max_score(query_word, verse_text, model_ksucca)
    0.5462957
    >>> model.similarity("الصلاة", "صلاة")
    0.5462957
    >>> model.similarity("الصلاة", "القيام")
    0.24506283
    
    @param query_word: only one query word
    @param verse_text: verse text (one or more words)
    @type query_word, verse_text: str (arabic unicode)
    @param model: the pretrained model to use
    @type model: KeyedVectors or Word2Vec
    @return: max similarity
    @rtype: float
    '''
    
    maxi = -1.0
    for verse_word in verse_text:
        if query_word not in model or verse_word not in model:
            continue
        score = model.similarity(query_word, verse_word)
        maxi = max(score, maxi)
    
    return max(0.0 , maxi)

In [ ]:
query_word = clean("الصلاة", 'FatText')[0]
verse_text = clean("صلاة القيام", 'FatText')
a = get_verse_max_score(query_word, verse_text, model_fasttext)
b = [model_fasttext.similarity(query_word, verse_text[0]), model_fasttext.similarity(query_word, verse_text[1])]
[a, b]

[0.6911777, [0.6911777, 0.40548423]]

In [33]:
a = [get_verse_max_score("صلاة", "صلاة القيام", model_ksucca), \
get_verse_max_score(clean("صلاة", 'KSUCCA')[0], clean("صلاة القيام", 'KSUCCA'), model_ksucca)]

b = [get_verse_max_score("صلاة", "صلاة القيام", model_tw), \
get_verse_max_score(clean("صلاة", 'TWITTER')[0], clean("صلاة القيام", 'TWITTER'), model_tw)]

c = [get_verse_max_score("صلاة", "صلاة القيام", model_fasttext), \
get_verse_max_score(clean("صلاة", 'FASTTEXT')[0], clean("صلاة القيام", 'FASTTEXT'), model_fasttext)]
[a, b, c]

[[0.042852506, 1.0], [0.0, 1.0], [0.04033663, 1.0]]

### Max Frequency for a 0.3 Similarity Score

By experiments, we specified a high similarity (0.3) between two words, returning the frequency score, that is the similarity score higher than 0.3, the more frequent the word should be in the verse.

In [36]:
def get_verse_frequency_score(query_word, verse_text, model):
    '''
    Get the frequency of a query word in a verse with:
    the similarity score higher than 0.3, the more frequent the word is in the verse.
    Example:
    >>> query_word = clean("الصلاة", 'KSUCCA')[0]
    >>> verse_text = clean("صلاة القيام", 'KSUCCA')
    >>> get_verse_frequency_score(query_word, verse_text, model_ksucca)
    2
    
    @param query_word: only one query word
    @param verse_text: verse text (one or more words)
    @type query_word, verse_text: str (arabic unicode)
    @param model: the pretrained model to use
    @type model: KeyedVectors or Word2Vec
    @return: frequency score
    @rtype: int
    '''

    freq = 0
    for verse_word in verse_text:
        if query_word not in model or verse_word not in model:
            continue
        score = model.similarity(query_word, verse_word)
        if score > 0.3:
            freq += 1

    return freq

In [37]:
get_verse_frequency_score(clean("الصلاة", 'KSUCCA')[0], clean("صلاة القيام", 'KSUCCA'), model_ksucca)

2

### Max Average Similarity Score

We made an average of Word2Vec vectors between two words (A word in a query and a word in a Verse): s(word, phrase) = sum(s(word, phrase[0]), .., s(word, phrase[n - 1])) / n 

Example:
```
similarity(word, word) = 1.0
similarity(phrase[0], word) = 0.234
similarity(phrase[1], word) = 0.012
..
vector(phrase) = [0.234, 0.012, 0.5654]
so, len(vector(phrase)) = 3

then, similarity(word, vector(phrase)) = AVG = (0.234 + 0.012 + 0.5654) / 3
```

In [39]:
def get_verse_avg_score(query_word, verse_text, model):
    '''
    Get the average similarity of query word and each word in a verse text.
    Example:
    >>> query_word = "الصلاة"
    >>> verse_text = "صلاة القيام"
    >>> get_avg_score(query_word, verse_text, model_ksucca)
    0.463957015
    >>> (0.64260626 + 0.330228) / 2.0
    0.463957015
    
    @param query_word: only one query word
    @param verse_text: verse text (one or more words)
    @type query_word, verse_text: str (arabic unicode)
    @param model: the pretrained model to use
    @type model: KeyedVectors or Word2Vec
    @return: average similarity
    @rtype: float
    '''

    verse_vector = []
    for verse_word in verse_text:
        if query_word not in model or verse_word not in model:
            continue
        score = model.similarity(query_word, verse_word)

        # Ignore negative scores
        if score > 0:
            verse_vector.append(score)

    # Avoid division by zero
    if len(verse_vector) == 0:
        return 0.0

    avg = sum(verse_vector) / len(verse_vector)
    return avg

In [40]:
query_word = clean("الصلاة", 'FatText')[0]
verse_text = clean("صلاة القيام", 'FatText')
a = get_verse_avg_score(query_word, verse_text, model_fasttext)
b = [model_fasttext.similarity(query_word, verse_text[0]), model_fasttext.similarity(query_word, verse_text[1])]
[a, b]

[0.5483309775590897, [0.6911777, 0.40548423]]

In [41]:
a = [get_verse_avg_score("الصلاة", "صلاة القيام", model_tw), \
get_verse_avg_score(clean("الصلاة", 'TWITTER')[0], clean("صلاة القيام", 'TWITTER'), model_tw)]

b = [get_verse_avg_score("الصلاة", "صلاة القيام", model_wiki), \
get_verse_avg_score(clean("الصلاة", 'WIKI')[0], clean("صلاة القيام", 'WIKI'), model_wiki)]

c = [get_verse_avg_score("الصلاة", "صلاة القيام", model_ksucca), \
get_verse_avg_score(clean("الصلاة", 'KSUCCA')[0], clean("صلاة القيام", 'KSUCCA'), model_ksucca)]

d = [get_verse_avg_score("الصلاة", "صلاة القيام", model_fasttext), \
get_verse_avg_score(clean("الصلاة", 'FASTTEXT')[0], clean("صلاة القيام", 'FASTTEXT'), model_fasttext)]

[a, b, c, d]

[[0.0, 0.6114272028207779],
 [0.0, 0.5175802633166313],
 [0.008036648738197982, 0.4639570116996765],
 [0.09752023965120316, 0.5483309775590897]]

## Most Similar Verses by a Query Word

Retrieve the most similar verses to a query word, according to one model and one of the 3 maximizing methods (max, freq, avg).

In [44]:
def get_most_similar_verses_by_query_word(query_word, model, method):
    '''
    Get the most similar verses to a query word,
    according to one model and one of the 3 maximizing methods (max, freq, avg).
    Example:
    >>> get_most_similar_verses_by_query_word(u"هدوء", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[0]
   (0.28051459789276123, 3896, 'سلام على إبراهيم')

    @param query_word: only one query word  
    @type query_word: str (arabic unicode)
    @param model: tuple of the pretrained model to use and its name
    @type model: KeyedVectors or Word2Vec
    @param method: the maximizing method to use
    @type method: function
    @return: most similar verses
    @rtype: list of tuples (score, verse_id, verse_text)
    '''

    (model_vectors, model_name) = model
    verse_props, verse_id = [], 0

    query_word = clean(query_word, model_name)
    if len(query_word):
      query_word = query_word[0]
    else:
      query_word = ''

    for verse in quran_clean_text:
        # Tokenizing and cleaning are made only once here :)
        score = method(query_word, clean(verse, model_name), model_vectors)
        verse_props.append((score, verse_id))
        verse_id += 1

    verse_props.sort(reverse=True)

    # Return at most 50 verses
    max_out_length = min(len(verse_props), 50)
    most_similar_verses = [(score, verse_id, quran_clean_text[verse_id])
                           for score, verse_id in verse_props[:max_out_length]]
    return most_similar_verses

## Testing on Quran by the loaded models

### AraVec cbow_300_wiki

In [49]:
get_most_similar_verses_by_query_word("هدوء", (model_wiki, 'WIKI'), get_verse_max_score)[:10]

[(0.53681993, 3693, 'وقالوا الحمد لله الذي أذهب عنا الحزن إن ربنا لغفور شكور'),
 (0.53681993,
  1679,
  'وتولى عنهم وقال يا أسفى على يوسف وابيضت عيناه من الحزن فهو كظيم'),
 (0.5333835,
  80,
  'ثم قست قلوبكم من بعد ذلك فهي كالحجارة أو أشد قسوة وإن من الحجارة لما يتفجر منه الأنهار وإن منها لما يشقق فيخرج منه الماء وإن منها لما يهبط من خشية الله وما الله بغافل عما تعملون'),
 (0.5307898,
  3261,
  'وأصبح فؤاد أم موسى فارغا إن كادت لتبدي به لولا أن ربطنا على قلبها لتكون من المؤمنين'),
 (0.5268251,
  1767,
  'مثل الذين كفروا بربهم أعمالهم كرماد اشتدت به الريح في يوم عاصف لا يقدرون مما كسبوا على شيء ذلك هو الضلال البعيد'),
 (0.5268251,
  1385,
  'هو الذي يسيركم في البر والبحر حتى إذا كنتم في الفلك وجرين بهم بريح طيبة وفرحوا بها جاءتها ريح عاصف وجاءهم الموج من كل مكان وظنوا أنهم أحيط بهم دعوا الله مخلصين له الدين لئن أنجيتنا من هذه لنكونن من الشاكرين'),
 (0.52408904, 6196, 'الذي أطعمهم من جوع وآمنهم من خوف'),
 (0.52408904, 5973, 'لا يسمن ولا يغني من جوع'),
 (0.505594,
  2585,
  'لا يحزنهم الف

In [50]:
get_most_similar_verses_by_query_word("هدوء", (model_wiki, 'WIKI'), get_verse_frequency_score)[:10]

[(6,
  1390,
  'والذين كسبوا السيئات جزاء سيئة بمثلها وترهقهم ذلة ما لهم من الله من عاصم كأنما أغشيت وجوههم قطعا من الليل مظلما أولئك أصحاب النار هم فيها خالدون'),
 (5,
  4588,
  'ويعذب المنافقين والمنافقات والمشركين والمشركات الظانين بالله ظن السوء عليهم دائرة السوء وغضب الله عليهم ولعنهم وأعد لهم جهنم وساءت مصيرا'),
 (5,
  3280,
  'فلما قضى موسى الأجل وسار بأهله آنس من جانب الطور نارا قال لأهله امكثوا إني آنست نارا لعلي آتيكم منها بخبر أو جذوة من النار لعلكم تصطلون'),
 (5,
  3168,
  'وألق عصاك فلما رآها تهتز كأنها جان ولى مدبرا ولم يعقب يا موسى لا تخف إني لا يخاف لدي المرسلون'),
 (5,
  2853,
  'لا تجعلوا دعاء الرسول بينكم كدعاء بعضكم بعضا قد يعلم الله الذين يتسللون منكم لواذا فليحذر الذين يخالفون عن أمره أن تصيبهم فتنة أو يصيبهم عذاب أليم'),
 (5,
  1723,
  'أنزل من السماء ماء فسالت أودية بقدرها فاحتمل السيل زبدا رابيا ومما يوقدون عليه في النار ابتغاء حلية أو متاع زبد مثله كذلك يضرب الله الحق والباطل فأما الزبد فيذهب جفاء وأما ما ينفع الناس فيمكث في الأرض كذلك يضرب الله الأمثال'),
 (5

In [51]:
get_most_similar_verses_by_query_word("الصواريخ", (model_wiki, 'WIKI'), get_verse_avg_score)[:10]

[(0.2994438757499059, 5913, 'النار ذات الوقود'),
 (0.298277348279953, 4981, 'خافضة رافعة'),
 (0.2895754352211952, 3354, 'فأنجيناه وأصحاب السفينة وجعلناها آية للعالمين'),
 (0.26430097222328186, 5677, 'ألم نجعل الأرض مهادا'),
 (0.26430097222328186, 5646, 'ألم نجعل الأرض كفاتا'),
 (0.26290107518434525, 5434, 'وجعل القمر فيهن نورا وجعل الشمس سراجا'),
 (0.24571284651756287, 3060, 'وتتخذون مصانع لعلكم تخلدون'),
 (0.2249307706952095, 5937, 'يخرج من بين الصلب والترائب'),
 (0.22046978771686554, 3059, 'أتبنون بكل ريع آية تعبثون'),
 (0.2126622125506401, 6191, 'ترميهم بحجارة من سجيل')]

### AraVec cbow_300_twitter

In [54]:
get_most_similar_verses_by_query_word("توبة", (model_tw, 'TWITTER'), get_verse_max_score)[:10]

[(1.0,
  5236,
  'يا أيها الذين آمنوا توبوا إلى الله توبة نصوحا عسى ربكم أن يكفر عنكم سيئاتكم ويدخلكم جنات تجري من تحتها الأنهار يوم لا يخزي الله النبي والذين آمنوا معه نورهم يسعى بين أيديهم وبأيمانهم يقولون ربنا أتمم لنا نورنا واغفر لنا إنك على كل شيء قدير'),
 (1.0,
  584,
  'وما كان لمؤمن أن يقتل مؤمنا إلا خطأ ومن قتل مؤمنا خطأ فتحرير رقبة مؤمنة ودية مسلمة إلى أهله إلا أن يصدقوا فإن كان من قوم عدو لكم وهو مؤمن فتحرير رقبة مؤمنة وإن كان من قوم بينكم وبينهم ميثاق فدية مسلمة إلى أهله وتحرير رقبة مؤمنة فمن لم يجد فصيام شهرين متتابعين توبة من الله وكان الله عليما حكيما'),
 (0.50092214,
  4296,
  'وهو الذي يقبل التوبة عن عباده ويعفو عن السيئات ويعلم ما تفعلون'),
 (0.50092214,
  1338,
  'ألم يعلموا أن الله هو يقبل التوبة عن عباده ويأخذ الصدقات وأن الله هو التواب الرحيم'),
 (0.50092214,
  510,
  'وليست التوبة للذين يعملون السيئات حتى إذا حضر أحدهم الموت قال إني تبت الآن ولا الذين يموتون وهم كفار أولئك أعتدنا لهم عذابا أليما'),
 (0.50092214,
  509,
  'إنما التوبة على الله للذين يعملون السوء ب

In [55]:
get_most_similar_verses_by_query_word("هدوء", (model_tw, 'TWITTER'), get_verse_max_score)[:10]

[(0.43591708,
  1259,
  'لقد نصركم الله في مواطن كثيرة ويوم حنين إذ أعجبتكم كثرتكم فلم تغن عنكم شيئا وضاقت عليكم الأرض بما رحبت ثم وليتم مدبرين'),
 (0.40902033, 1905, 'والأنعام خلقها لكم فيها دفء ومنافع ومنها تأكلون'),
 (0.40315998,
  4613,
  'يا أيها الذين آمنوا لا ترفعوا أصواتكم فوق صوت النبي ولا تجهروا له بالقول كجهر بعضكم لبعض أن تحبط أعمالكم وأنتم لا تشعرون'),
 (0.39123854, 1792, 'مهطعين مقنعي رءوسهم لا يرتد إليهم طرفهم وأفئدتهم هواء'),
 (0.3648635, 3216, 'وأمطرنا عليهم مطرا فساء مطر المنذرين'),
 (0.3648635, 3104, 'وأمطرنا عليهم مطرا فساء مطر المنذرين'),
 (0.3648635,
  2894,
  'ولقد أتوا على القرية التي أمطرت مطر السوء أفلم يكونوا يرونها بل كانوا لا يرجون نشورا'),
 (0.3648635,
  594,
  'وإذا كنت فيهم فأقمت لهم الصلاة فلتقم طائفة منهم معك وليأخذوا أسلحتهم فإذا سجدوا فليكونوا من ورائكم ولتأت طائفة أخرى لم يصلوا فليصلوا معك وليأخذوا حذرهم وأسلحتهم ود الذين كفروا لو تغفلون عن أسلحتكم وأمتعتكم فيميلون عليكم ميلة واحدة ولا جناح عليكم إن كان بكم أذى من مطر أو كنتم مرضى أن تضعوا أسلحتكم و

In [56]:
get_most_similar_verses_by_query_word("الراحة", (model_tw, 'TWITTER'), get_verse_frequency_score)[:10]

[(5,
  1732,
  'الله يبسط الرزق لمن يشاء ويقدر وفرحوا بالحياة الدنيا وما الحياة الدنيا في الآخرة إلا متاع'),
 (4,
  5094,
  'اعلموا أنما الحياة الدنيا لعب ولهو وزينة وتفاخر بينكم وتكاثر في الأموال والأولاد كمثل غيث أعجب الكفار نباته ثم يهيج فتراه مصفرا ثم يكون حطاما وفي الآخرة عذاب شديد ومغفرة من الله ورضوان وما الحياة الدنيا إلا متاع الغرور'),
 (4,
  3551,
  'أشحة عليكم فإذا جاء الخوف رأيتهم ينظرون إليك تدور أعينهم كالذي يغشى عليه من الموت فإذا ذهب الخوف سلقوكم بألسنة حداد أشحة على الخير أولئك لم يؤمنوا فأحبط الله أعمالهم وكان ذلك على الله يسيرا'),
 (4,
  1272,
  'يا أيها الذين آمنوا ما لكم إذا قيل لكم انفروا في سبيل الله اثاقلتم إلى الأرض أرضيتم بالحياة الدنيا من الآخرة فما متاع الحياة الدنيا في الآخرة إلا قليل'),
 (4,
  774,
  'يا أيها الذين آمنوا شهادة بينكم إذا حضر أحدكم الموت حين الوصية اثنان ذوا عدل منكم أو آخران من غيركم إن أنتم ضربتم في الأرض فأصابتكم مصيبة الموت تحبسونهما من بعد الصلاة فيقسمان بالله إن ارتبتم لا نشتري به ثمنا ولو كان ذا قربى ولا نكتم شهادة الله إنا إذا لمن ال

In [52]:
get_most_similar_verses_by_query_word("هدوء", (model_tw, 'TWITTER'), get_verse_avg_score)[:10]

[(0.35617557168006897, 5681, 'وجعلنا الليل لباسا'),
 (0.3175879716873169, 6084, 'ألم يجدك يتيما فآوى'),
 (0.26131783922513324, 5022, 'لا بارد ولا كريم'),
 (0.25994470715522766, 6061, 'إن سعيكم لشتى'),
 (0.25994470715522766, 4456, 'إن شجرت الزقوم'),
 (0.25994470715522766, 3891, 'وناديناه أن يا إبراهيم'),
 (0.25994470715522766, 3791, 'إن إلهكم لواحد'),
 (0.25994470715522766, 2550, 'قالوا حرقوه وانصروا آلهتكم إن كنتم فاعلين'),
 (0.25994470715522766,
  1447,
  'وقال موسى يا قوم إن كنتم آمنتم بالله فعليه توكلوا إن كنتم مسلمين'),
 (0.25622887909412384,
  2410,
  'قالوا إن هذان لساحران يريدان أن يخرجاكم من أرضكم بسحرهما ويذهبا بطريقتكم المثلى')]

### KSUCCA full_cbow

In [60]:
get_most_similar_verses_by_query_word("هدوء", (model_ksucca, 'KSUCCA'), get_verse_max_score)[:10]

[(0.33699578, 5680, 'وجعلنا نومكم سباتا'),
 (0.33699578,
  2901,
  'وهو الذي جعل لكم الليل لباسا والنوم سباتا وجعل النهار نشورا'),
 (0.33044893, 5817, 'والصبح إذا تنفس'),
 (0.3220397, 62, 'ثم بعثناكم من بعد موتكم لعلكم تشكرون'),
 (0.31106934,
  446,
  'ثم أنزل عليكم من بعد الغم أمنة نعاسا يغشى طائفة منكم وطائفة قد أهمتهم أنفسهم يظنون بالله غير الحق ظن الجاهلية يقولون هل لنا من الأمر من شيء قل إن الأمر كله لله يخفون في أنفسهم ما لا يبدون لك يقولون لو كان لنا من الأمر شيء ما قتلنا هاهنا قل لو كنتم في بيوتكم لبرز الذين كتب عليهم القتل إلى مضاجعهم وليبتلي الله ما في صدوركم وليمحص ما في قلوبكم والله عليم بذات الصدور'),
 (0.31055403, 6079, 'بسم الله الرحمن الرحيم والضحى'),
 (0.299146,
  261,
  'الله لا إله إلا هو الحي القيوم لا تأخذه سنة ولا نوم له ما في السماوات وما في الأرض من ذا الذي يشفع عنده إلا بإذنه يعلم ما بين أيديهم وما خلفهم ولا يحيطون بشيء من علمه إلا بما شاء وسع كرسيه السماوات والأرض ولا يئوده حفظهما وهو العلي العظيم'),
 (0.2896562,
  4267,
  'ولئن أذقناه رحمة منا من بعد ضراء مست

In [61]:
get_most_similar_verses_by_query_word("الوفاء", (model_ksucca, 'KSUCCA'), get_verse_frequency_score)[:10]

[(2, 4960, 'هل جزاء الإحسان إلا الإحسان'),
 (2, 2183, 'هنالك الولاية لله الحق هو خير ثوابا وخير عقبا'),
 (2,
  2062,
  'ولا تقربوا مال اليتيم إلا بالتي هي أحسن حتى يبلغ أشده وأوفوا بالعهد إن العهد كان مسئولا'),
 (2,
  1991,
  'وأوفوا بعهد الله إذا عاهدتم ولا تنقضوا الأيمان بعد توكيدها وقد جعلتم الله عليكم كفيلا إن الله يعلم ما تفعلون'),
 (2,
  487,
  'فاستجاب لهم ربهم أني لا أضيع عمل عامل منكم من ذكر أو أنثى بعضكم من بعض فالذين هاجروا وأخرجوا من ديارهم وأوذوا في سبيلي وقاتلوا وقتلوا لأكفرن عنهم سيئاتهم ولأدخلنهم جنات تجري من تحتها الأنهار ثوابا من عند الله والله عنده حسن الثواب'),
 (2,
  86,
  'وقالوا لن تمسنا النار إلا أياما معدودة قل أتخذتم عند الله عهدا فلن يخلف الله عهده أم تقولون على الله ما لا تعلمون'),
 (1, 6178, 'إلا الذين آمنوا وعملوا الصالحات وتواصوا بالحق وتواصوا بالصبر'),
 (1, 6039, 'ثم كان من الذين آمنوا وتواصوا بالصبر وتواصوا بالمرحمة'),
 (1, 5940, 'فما له من قوة ولا ناصر'),
 (1, 5749, 'وآثر الحياة الدنيا')]

In [62]:
get_most_similar_verses_by_query_word("هدوء", (model_ksucca, 'KSUCCA'), get_verse_frequency_score)[:10]

[(2, 2901, 'وهو الذي جعل لكم الليل لباسا والنوم سباتا وجعل النهار نشورا'),
 (1, 6079, 'بسم الله الرحمن الرحيم والضحى'),
 (1, 5817, 'والصبح إذا تنفس'),
 (1, 5680, 'وجعلنا نومكم سباتا'),
 (1,
  446,
  'ثم أنزل عليكم من بعد الغم أمنة نعاسا يغشى طائفة منكم وطائفة قد أهمتهم أنفسهم يظنون بالله غير الحق ظن الجاهلية يقولون هل لنا من الأمر من شيء قل إن الأمر كله لله يخفون في أنفسهم ما لا يبدون لك يقولون لو كان لنا من الأمر شيء ما قتلنا هاهنا قل لو كنتم في بيوتكم لبرز الذين كتب عليهم القتل إلى مضاجعهم وليبتلي الله ما في صدوركم وليمحص ما في قلوبكم والله عليم بذات الصدور'),
 (1, 62, 'ثم بعثناكم من بعد موتكم لعلكم تشكرون'),
 (0, 6235, 'من الجنة والناس'),
 (0, 6234, 'الذي يوسوس في صدور الناس'),
 (0, 6233, 'من شر الوسواس الخناس'),
 (0, 6232, 'إله الناس')]

In [58]:
get_most_similar_verses_by_query_word("شجاعة", (model_ksucca, 'KSUCCA'), get_verse_avg_score)[:10]

[(0.2579765319824219, 3943, 'أم لكم سلطان مبين'),
 (0.2246875762939453, 4788, 'علمه شديد القوى'),
 (0.21949513256549835, 5910, 'واليوم الموعود'),
 (0.19739185273647308, 2262, 'وحنانا من لدنا وزكاة وكان تقيا'),
 (0.19041509181261063, 3410, 'غلبت الروم'),
 (0.1840280145406723, 5852, 'ليوم عظيم'),
 (0.1762766788403193, 5512, 'إنه فكر وقدر'),
 (0.17603401839733124, 4902, 'علم القرآن'),
 (0.17246948927640915, 5773, 'كرام بررة'),
 (0.16737867891788483, 5922, 'وهو الغفور الودود')]

In [59]:
get_most_similar_verses_by_query_word("هدوء", (model_ksucca, 'KSUCCA'), get_verse_avg_score)[:10]

[(0.2972675710916519, 5680, 'وجعلنا نومكم سباتا'),
 (0.20586979389190674, 5755, 'إلى ربك منتهاها'),
 (0.20282242695490518, 5902, 'لتركبن طبقا عن طبق'),
 (0.20207566767930984, 5733, 'ثم أدبر يسعى'),
 (0.20207566767930984, 5517, 'ثم أدبر واستكبر'),
 (0.1981704756617546, 5713, 'والناشطات نشطا'),
 (0.19686564803123474, 5714, 'والسابحات سبحا'),
 (0.19665132462978363, 5951, 'والذي أخرج المرعى'),
 (0.19460177421569824, 5817, 'والصبح إذا تنفس'),
 (0.19293488065401712, 5816, 'والليل إذا عسعس')]

### FastText Arabic CBOW-300

In [70]:
get_most_similar_verses_by_query_word("الوفاء", (model_fasttext, 'FASTTEXT'), get_verse_max_score)[:10]

[(0.50314784, 5265, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.50314784, 3752, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.50314784, 3634, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.50314784, 3229, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.50314784,
  2579,
  'واقترب الوعد الحق فإذا هي شاخصة أبصار الذين كفروا يا ويلنا قد كنا في غفلة من هذا بل كنا ظالمين'),
 (0.50314784, 2520, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.50314784, 2491, 'ثم صدقناهم الوعد فأنجيناهم ومن نشاء وأهلكنا المسرفين'),
 (0.50314784,
  2303,
  'واذكر في الكتاب إسماعيل إنه كان صادق الوعد وكان رسولا نبيا'),
 (0.50314784, 1411, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.4954666,
  4525,
  'أولئك الذين نتقبل عنهم أحسن ما عملوا ونتجاوز عن سيئاتهم في أصحاب الجنة وعد الصدق الذي كانوا يوعدون')]

In [71]:
get_most_similar_verses_by_query_word("هدوء", (model_fasttext, 'FASTTEXT'), get_verse_max_score)[:10]

[(0.46643838,
  563,
  'يا أيها الذين آمنوا خذوا حذركم فانفروا ثبات أو انفروا جميعا'),
 (0.4216647, 4005, 'فسخرنا له الريح تجري بأمره رخاء حيث أصاب'),
 (0.41341564, 4663, 'ادخلوها بسلام ذلك يوم الخلود'),
 (0.41341564, 1847, 'ادخلوها بسلام آمنين'),
 (0.41341564,
  1520,
  'قيل يا نوح اهبط بسلام منا وبركات عليك وعلى أمم ممن معك وأمم سنمتعهم ثم يمسهم منا عذاب أليم'),
 (0.41340402, 3968, 'وسلام على المرسلين'),
 (0.41340402,
  3217,
  'قل الحمد لله وسلام على عباده الذين اصطفى آلله خير أما يشركون'),
 (0.41340402, 2264, 'وسلام عليه يوم ولد ويوم يموت ويوم يبعث حيا'),
 (0.3914, 6129, 'سلام هي حتى مطلع الفجر'),
 (0.3914, 4699, 'إذ دخلوا عليه فقالوا سلاما قال سلام قوم منكرون')]

In [72]:
get_most_similar_verses_by_query_word("هدوء", (model_fasttext, 'FASTTEXT'), get_verse_frequency_score)[:10]

[(2,
  5260,
  'أمن هذا الذي هو جند لكم ينصركم من دون الرحمن إن الكافرون إلا في غرور'),
 (2,
  4608,
  'إذ جعل الذين كفروا في قلوبهم الحمية حمية الجاهلية فأنزل الله سكينته على رسوله وعلى المؤمنين وألزمهم كلمة التقوى وكانوا أحق بها وأهلها وكان الله بكل شيء عليما'),
 (2,
  4544,
  'فاصبر كما صبر أولو العزم من الرسل ولا تستعجل لهم كأنهم يوم يرون ما يوعدون لم يلبثوا إلا ساعة من نهار بلاغ فهل يهلك إلا القوم الفاسقون'),
 (2,
  4287,
  'والذين يحاجون في الله من بعد ما استجيب له حجتهم داحضة عند ربهم وعليهم غضب ولهم عذاب شديد'),
 (2,
  3356,
  'إنما تعبدون من دون الله أوثانا وتخلقون إفكا إن الذين تعبدون من دون الله لا يملكون لكم رزقا فابتغوا عند الله الرزق واعبدوه واشكروا له إليه ترجعون'),
 (2,
  3218,
  'أمن خلق السماوات والأرض وأنزل لكم من السماء ماء فأنبتنا به حدائق ذات بهجة ما كان لكم أن تنبتوا شجرها أإله مع الله بل هم قوم يعدلون'),
 (2,
  2848,
  'يا أيها الذين آمنوا ليستأذنكم الذين ملكت أيمانكم والذين لم يبلغوا الحلم منكم ثلاث مرات من قبل صلاة الفجر وحين تضعون ثيابكم من الظهيرة ومن بعد صل

In [63]:
get_most_similar_verses_by_query_word("هدوء", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[:10]

[(0.28051459789276123, 3896, 'سلام على إبراهيم'),
 (0.27566253145535785, 1847, 'ادخلوها بسلام آمنين'),
 (0.26370637615521747, 3917, 'سلام على إل ياسين'),
 (0.2635757140815258, 6129, 'سلام هي حتى مطلع الفجر'),
 (0.25783757865428925, 3907, 'سلام على موسى وهارون'),
 (0.2575249100724856, 3968, 'وسلام على المرسلين'),
 (0.24845048040151596, 3471, 'هدى ورحمة للمحسنين'),
 (0.24728529155254364, 1065, 'يأتوك بكل ساحر عليم'),
 (0.2468289166688919, 4663, 'ادخلوها بسلام ذلك يوم الخلود'),
 (0.2376958653330803, 5008, 'وظل ممدود')]

In [65]:
get_most_similar_verses_by_query_word("التهجد", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[:10]

[(0.3283767029643059, 4669, 'ومن الليل فسبحه وأدبار السجود'),
 (0.31168246269226074, 5995, 'والشفع والوتر'),
 (0.2832338586449623, 4783, 'ومن الليل فسبحه وإدبار النجوم'),
 (0.2804764628942524,
  2106,
  'أقم الصلاة لدلوك الشمس إلى غسق الليل وقرآن الفجر إن قرآن الفجر كان مشهودا'),
 (0.27031098554531735,
  309,
  'الصابرين والصادقين والقانتين والمنفقين والمستغفرين بالأسحار'),
 (0.266217365860939, 5816, 'والليل إذا عسعس'),
 (0.25197137147188187, 4902, 'علم القرآن'),
 (0.24958699196577072, 5396, 'إلا المصلين'),
 (0.2486003413796425,
  244,
  'حافظوا على الصلوات والصلاة الوسطى وقوموا لله قانتين'),
 (0.24542429558932782, 2502, 'يسبحون الليل والنهار لا يفترون')]

In [67]:
get_most_similar_verses_by_query_word("طمأنينة", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[:10]

[(0.3177646646896998, 6019, 'يا أيتها النفس المطمئنة'),
 (0.29770080745220184, 1847, 'ادخلوها بسلام آمنين'),
 (0.26301364600658417, 3833, 'بيضاء لذة للشاربين'),
 (0.25496935844421387, 5072, 'وتصلية جحيم'),
 (0.25412189215421677, 6196, 'الذي أطعمهم من جوع وآمنهم من خوف'),
 (0.25089242309331894, 5796, 'ضاحكة مستبشرة'),
 (0.2484331081310908, 3021, 'وأزلفت الجنة للمتقين'),
 (0.24673651903867722, 5720, 'أبصارها خاشعة'),
 (0.246241707354784, 5004, 'إلا قيلا سلاما سلاما'),
 (0.24427675704161325, 3968, 'وسلام على المرسلين')]

In [68]:
get_most_similar_verses_by_query_word("هدوء", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[:10]

[(0.28051459789276123, 3896, 'سلام على إبراهيم'),
 (0.27566253145535785, 1847, 'ادخلوها بسلام آمنين'),
 (0.26370637615521747, 3917, 'سلام على إل ياسين'),
 (0.2635757140815258, 6129, 'سلام هي حتى مطلع الفجر'),
 (0.25783757865428925, 3907, 'سلام على موسى وهارون'),
 (0.2575249100724856, 3968, 'وسلام على المرسلين'),
 (0.24845048040151596, 3471, 'هدى ورحمة للمحسنين'),
 (0.24728529155254364, 1065, 'يأتوك بكل ساحر عليم'),
 (0.2468289166688919, 4663, 'ادخلوها بسلام ذلك يوم الخلود'),
 (0.2376958653330803, 5008, 'وظل ممدود')]

### General Testing

In [73]:
query_word = u'الطاعة'
verse_text = 'يَا أَيُّهَا الَّذِينَ آمَنُوا أَطِيعُوا اللَّهَ وَأَطِيعُوا الرَّسُولَ وَأُولِي الْأَمْرِ مِنْكُمْ فَإِنْ تَنَازَعْتُمْ فِي شَيْءٍ فَرُدُّوهُ إِلَى اللَّهِ وَالرَّسُولِ إِنْ كُنْتُمْ تُؤْمِنُونَ بِاللَّهِ وَالْيَوْمِ الْآخِرِ ذَلِكَ خَيْرٌ وَأَحْسَنُ تَأْوِيلًا'

maxi = [get_verse_max_score(clean(query_word, 'WIKI')[0], verse_text, model_wiki),
        get_verse_max_score(clean(query_word, 'TWITTER')[0], verse_text, model_tw),
        get_verse_max_score(clean(query_word, 'KSUCCA')[0], verse_text, model_ksucca),
        get_verse_max_score(clean(query_word, 'FASTTEXT')[0], verse_text, model_fasttext)]

freq = [get_verse_frequency_score(clean(query_word, 'WIKI')[0], verse_text, model_wiki),
        get_verse_frequency_score(clean(query_word, 'TWITTER')[0], verse_text, model_tw),
        get_verse_frequency_score(clean(query_word, 'KSUCCA')[0], verse_text, model_ksucca),
        get_verse_frequency_score(clean(query_word, 'FASTTEXT')[0], verse_text, model_fasttext)]

avg = [get_verse_avg_score(clean(query_word, 'WIKI')[0], verse_text, model_wiki),
       get_verse_avg_score(clean(query_word, 'TWITTER')[0], verse_text, model_tw),
       get_verse_avg_score(clean(query_word, 'KSUCCA')[0], verse_text, model_ksucca),
       get_verse_avg_score(clean(query_word, 'FASTTEXT')[0], verse_text, model_fasttext)]

[verse_text, [maxi, freq, avg]]

['يَا أَيُّهَا الَّذِينَ آمَنُوا أَطِيعُوا اللَّهَ وَأَطِيعُوا الرَّسُولَ وَأُولِي الْأَمْرِ مِنْكُمْ فَإِنْ تَنَازَعْتُمْ فِي شَيْءٍ فَرُدُّوهُ إِلَى اللَّهِ وَالرَّسُولِ إِنْ كُنْتُمْ تُؤْمِنُونَ بِاللَّهِ وَالْيَوْمِ الْآخِرِ ذَلِكَ خَيْرٌ وَأَحْسَنُ تَأْوِيلًا',
 [[0.23093556, 0.05757772, 0.087316565, 0.07691681],
  [0, 0, 0, 0],
  [0.0844209166450633,
   0.029759011818896397,
   0.03572694238807474,
   0.058686121233872006]]]

# Sentence Embedding

```python
query = "صلاة قيام الليل"
```
Important notes:
  * In AraVec Twitter model, it is trained on tweets dataset, and by experiments we noticed that they had not removed the underscore character '_' while scraping tweets with hashtags, so a query_word or a verse_word may be one "صلاة" or two "صلاة_القيام".


## Most Similar Verses by a Query Text

In [74]:
from pyarabic.araby import tokenize

def get_most_similar_verses_by_query_text(query_text, model, method):
    '''
    Get the most similar verses to a query text,
    according to one model and one of the 3 maximizing methods (max, freq, avg).
    Example:
    >>> query_text = clean("مثوى الكافرين", ‘KSUCCA’)[0]
    >>> get_most_similar_verses_by_query_text(query_text, (model_ksucca, 'KSUCCA'), get_verse_avg_score)[:5]
    [(0.46887486428022385, 4025, 'جهنم يصلونها فبئس المهاد'),
     (0.4668786823749542, 5693, 'للطاغين مآبا'),
     (0.4514587353914976, 1929, 'فادخلوا أبواب جهنم خالدين فيها فلبئس مثوى المتكبرين'),
     (0.448849493637681, 4208, 'ادخلوا أبواب جهنم خالدين فيها فبئس مثوى المتكبرين'),
     (0.4369580075144768, 1778, 'جهنم يصلونها وبئس القرار')]

    @param query_text: query text (one or more words)
    @type query_text: str (arabic unicode)
    @param model: tuple of the pretrained model to use and its name
    @type model: KeyedVectors or Word2Vec
    @param method: the maximizing method to use
    @type method: function
    @return: most similar verses
    @rtype: list of tuples (score, verse_id, verse_text)
    '''

    # Generalized and better than the split method
    query_text = tokenize(query_text)

    verse2score = {}
    for i in range(len(query_text)):
        most_similar_verses1 = get_most_similar_verses_by_query_word(query_text[i], model, method)
        for score, verse_id, verse in most_similar_verses1:
            # Doubling the score for the frequent results
            if (verse_id , verse) in verse2score:
                verse2score[(verse_id, verse)] += score
            else:
                verse2score[(verse_id, verse)] = score

        # Feature: Comparing a query word of one 'صلاة' or two concatenated words 'صلاة_القيام',
        # with a verse word of one or two concatenated words
        # WARNING: Works only with AraVec twitter model
        if model[1] == 'TWITTER' and i + 1 < len(query_text):
            term = query_text[i] + "_" + query_text[i + 1]
            most_similar_verses2 = get_most_similar_verses_by_query_word(term, model, method)
            for score, verse_id, verse in most_similar_verses2:
                if (verse_id, verse) in verse2score:
                    verse2score[(verse_id, verse)] += score
                else:
                    verse2score[(verse_id, verse)] = score

    best_verses = [(score, verse_id, verse) for (verse_id, verse), score in verse2score.items()]
    best_verses.sort(reverse=True)

    # Return at most 50 verses
    max_out_length = min(len(best_verses), 50)
    return best_verses[:max_out_length]

### Testing version 1

In [89]:
get_most_similar_verses_by_query_text("التهجد وقيام الليل", (model_wiki, 'WIKI'), get_verse_max_score)[:10]

[(1.5949079,
  1553,
  'قالوا يا لوط إنا رسل ربك لن يصلوا إليك فأسر بأهلك بقطع من الليل ولا يلتفت منكم أحد إلا امرأتك إنه مصيبها ما أصابهم إن موعدهم الصبح أليس الصبح بقريب'),
 (1.0592327,
  223,
  'يسألونك عن الشهر الحرام قتال فيه قل قتال فيه كبير وصد عن سبيل الله وكفر به والمسجد الحرام وإخراج أهله منه أكبر عند الله والفتنة أكبر من القتل ولا يزالون يقاتلونكم حتى يردوكم عن دينكم إن استطاعوا ومن يرتدد منكم عن دينه فيمت وهو كافر فأولئك حبطت أعمالهم في الدنيا والآخرة وأولئك أصحاب النار هم فيها خالدون'),
 (1.0, 5681, 'وجعلنا الليل لباسا'),
 (1.0, 5616, 'ومن الليل فاسجد له وسبحه ليلا طويلا'),
 (1.0,
  5494,
  'إن ربك يعلم أنك تقوم أدنى من ثلثي الليل ونصفه وثلثه وطائفة من الذين معك والله يقدر الليل والنهار علم أن لن تحصوه فتاب عليكم فاقرءوا ما تيسر من القرآن علم أن سيكون منكم مرضى وآخرون يضربون في الأرض يبتغون من فضل الله وآخرون يقاتلون في سبيل الله فاقرءوا ما تيسر منه وأقيموا الصلاة وآتوا الزكاة وأقرضوا الله قرضا حسنا وما تقدموا لأنفسكم من خير تجدوه عند الله هو خيرا وأعظم أجرا واستغفروا الله

In [87]:
get_most_similar_verses_by_query_text('صفات المصلين', (model_tw, 'TWITTER'), get_verse_max_score)[:10]

[(1.0, 5537, 'قالوا لم نك من المصلين'),
 (1.0, 5396, 'إلا المصلين'),
 (0.53869915, 5464, 'وأن المساجد لله فلا تدعوا مع الله أحدا'),
 (0.53869915,
  193,
  'أحل لكم ليلة الصيام الرفث إلى نسائكم هن لباس لكم وأنتم لباس لهن علم الله أنكم كنتم تختانون أنفسكم فتاب عليكم وعفا عنكم فالآن باشروهن وابتغوا ما كتب الله لكم وكلوا واشربوا حتى يتبين لكم الخيط الأبيض من الخيط الأسود من الفجر ثم أتموا الصيام إلى الليل ولا تباشروهن وأنتم عاكفون في المساجد تلك حدود الله فلا تقربوها كذلك يبين الله آياته للناس لعلهم يتقون'),
 (0.5083466,
  4609,
  'لقد صدق الله رسوله الرؤيا بالحق لتدخلن المسجد الحرام إن شاء الله آمنين محلقين رءوسكم ومقصرين لا تخافون فعلم ما لم تعلموا فجعل من دون ذلك فتحا قريبا'),
 (0.5083466,
  4607,
  'هم الذين كفروا وصدوكم عن المسجد الحرام والهدي معكوفا أن يبلغ محله ولولا رجال مؤمنون ونساء مؤمنات لم تعلموهم أن تطئوهم فتصيبكم منهم معرة بغير علم ليدخل الله في رحمته من يشاء لو تزيلوا لعذبنا الذين كفروا منهم عذابا أليما'),
 (0.5083466,
  2035,
  'إن أحسنتم أحسنتم لأنفسكم وإن أسأتم فلها فإذا 

In [91]:
get_most_similar_verses_by_query_text('التوبة', (model_ksucca, 'KSUCCA'), get_verse_avg_score)[:10]

[(0.30748512264755035,
  4296,
  'وهو الذي يقبل التوبة عن عباده ويعفو عن السيئات ويعلم ما تفعلون'),
 (0.28008219599723816, 5808, 'بأي ذنب قتلت'),
 (0.24939078092575073, 5016, 'لأصحاب اليمين'),
 (0.2432815883504717,
  509,
  'إنما التوبة على الله للذين يعملون السوء بجهالة ثم يتوبون من قريب فأولئك يتوب الله عليهم وكان الله عليما حكيما'),
 (0.2418473412593206, 5542, 'فما تنفعهم شفاعة الشافعين'),
 (0.24040193855762482, 1152, 'خذ العفو وأمر بالعرف وأعرض عن الجاهلين'),
 (0.2284503554304441, 4249, 'نزلا من غفور رحيم'),
 (0.22488315850496293,
  1338,
  'ألم يعلموا أن الله هو يقبل التوبة عن عباده ويأخذ الصدقات وأن الله هو التواب الرحيم'),
 (0.21767877042293549, 5806, 'وإذا النفوس زوجت'),
 (0.2175883710384369,
  2924,
  'إلا من تاب وآمن وعمل عملا صالحا فأولئك يبدل الله سيئاتهم حسنات وكان الله غفورا رحيما')]

In [97]:
get_most_similar_verses_by_query_text('الوفاء بالعهد', (model_ksucca, 'KSUCCA'), get_verse_frequency_score)[:10]

[(8,
  1991,
  'وأوفوا بعهد الله إذا عاهدتم ولا تنقضوا الأيمان بعد توكيدها وقد جعلتم الله عليكم كفيلا إن الله يعلم ما تفعلون'),
 (6,
  2062,
  'ولا تقربوا مال اليتيم إلا بالتي هي أحسن حتى يبلغ أشده وأوفوا بالعهد إن العهد كان مسئولا'),
 (6,
  46,
  'يا بني إسرائيل اذكروا نعمتي التي أنعمت عليكم وأوفوا بعهدي أوف بعهدكم وإياي فارهبون'),
 (4,
  2844,
  'قل أطيعوا الله وأطيعوا الرسول فإن تولوا فإنما عليه ما حمل وعليكم ما حملتم وإن تطيعوه تهتدوا وما على الرسول إلا البلاغ المبين'),
 (4,
  86,
  'وقالوا لن تمسنا النار إلا أياما معدودة قل أتخذتم عند الله عهدا فلن يخلف الله عهده أم تقولون على الله ما لا تعلمون'),
 (3,
  5112,
  'يا أيها الذين آمنوا إذا تناجيتم فلا تتناجوا بالإثم والعدوان ومعصيت الرسول وتناجوا بالبر والتقوى واتقوا الله الذي إليه تحشرون'),
 (3,
  2672,
  'وجاهدوا في الله حق جهاده هو اجتباكم وما جعل عليكم في الدين من حرج ملة أبيكم إبراهيم هو سماكم المسلمين من قبل وفي هذا ليكون الرسول شهيدا عليكم وتكونوا شهداء على الناس فأقيموا الصلاة وآتوا الزكاة واعتصموا بالله هو مولاكم فنعم المولى

In [90]:
get_most_similar_verses_by_query_text('الوفاء بالعهد', (model_ksucca, 'KSUCCA'), get_verse_max_score)[:10]

[(1.4854275,
  2062,
  'ولا تقربوا مال اليتيم إلا بالتي هي أحسن حتى يبلغ أشده وأوفوا بالعهد إن العهد كان مسئولا'),
 (0.98105514,
  2433,
  'فرجع موسى إلى قومه غضبان أسفا قال يا قوم ألم يعدكم ربكم وعدا حسنا أفطال عليكم العهد أم أردتم أن يحل عليكم غضب من ربكم فأخلفتم موعدي'),
 (0.8965465,
  1991,
  'وأوفوا بعهد الله إذا عاهدتم ولا تنقضوا الأيمان بعد توكيدها وقد جعلتم الله عليكم كفيلا إن الله يعلم ما تفعلون'),
 (0.82758605,
  4592,
  'إن الذين يبايعونك إنما يبايعون الله يد الله فوق أيديهم فمن نكث فإنما ينكث على نفسه ومن أوفى بما عاهد عليه الله فسيؤتيه أجرا عظيما'),
 (0.8007281,
  1345,
  'إن الله اشترى من المؤمنين أنفسهم وأموالهم بأن لهم الجنة يقاتلون في سبيل الله فيقتلون ويقتلون وعدا عليه حقا في التوراة والإنجيل والقرآن ومن أوفى بعهده من الله فاستبشروا ببيعكم الذي بايعتم به وذلك هو الفوز العظيم'),
 (0.8007281, 368, 'بلى من أوفى بعهده واتقى فإن الله يحب المتقين'),
 (0.75989807,
  86,
  'وقالوا لن تمسنا النار إلا أياما معدودة قل أتخذتم عند الله عهدا فلن يخلف الله عهده أم تقولون على الله ما

In [93]:
get_most_similar_verses_by_query_text('الوفاء بالعهد', (model_fasttext, 'FASTTEXT'), get_verse_max_score)[:10]

[(1.4327471,
  2062,
  'ولا تقربوا مال اليتيم إلا بالتي هي أحسن حتى يبلغ أشده وأوفوا بالعهد إن العهد كان مسئولا'),
 (0.9571479,
  1345,
  'إن الله اشترى من المؤمنين أنفسهم وأموالهم بأن لهم الجنة يقاتلون في سبيل الله فيقتلون ويقتلون وعدا عليه حقا في التوراة والإنجيل والقرآن ومن أوفى بعهده من الله فاستبشروا ببيعكم الذي بايعتم به وذلك هو الفوز العظيم'),
 (0.9571479, 368, 'بلى من أوفى بعهده واتقى فإن الله يحب المتقين'),
 (0.93902236, 5265, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.93902236, 3752, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.93902236, 3634, 'ويقولون متى هذا الوعد إن كنتم صادقين'),
 (0.92999005,
  183,
  'ليس البر أن تولوا وجوهكم قبل المشرق والمغرب ولكن البر من آمن بالله واليوم الآخر والملائكة والكتاب والنبيين وآتى المال على حبه ذوي القربى واليتامى والمساكين وابن السبيل والسائلين وفي الرقاب وأقام الصلاة وآتى الزكاة والموفون بعهدهم إذا عاهدوا والصابرين في البأساء والضراء وحين البأس أولئك الذين صدقوا وأولئك هم المتقون'),
 (0.91666865, 5597, 'يوفون بالنذر ويخافون يوما كان شر

In [84]:
get_most_similar_verses_by_query_text("علوم البلاغة", (model_tw, 'TWITTER'), get_verse_avg_score)[:10]

[(0.4950829322139422, 5056, 'في كتاب مكنون'),
 (0.48159336671233177, 4902, 'علم القرآن'),
 (0.45381811261177063, 5634, 'ليوم الفصل'),
 (0.29319538176059723, 6110, 'علم الإنسان ما لم يعلم'),
 (0.2927052676677704, 3, 'مالك يوم الدين'),
 (0.27782437205314636, 5048, 'لو نشاء جعلناه أجاجا فلولا تشكرون'),
 (0.2738831043243408, 4964, 'مدهامتان'),
 (0.26571905612945557, 5054, 'وإنه لقسم لو تعلمون عظيم'),
 (0.2630220055580139, 5867, 'كتاب مرقوم'),
 (0.2630220055580139, 5856, 'كتاب مرقوم')]

In [86]:
get_most_similar_verses_by_query_text("علوم البلاغة", (model_fasttext, 'FASTTEXT'), get_verse_avg_score)[:10]

[(0.8510092794895172, 4902, 'علم القرآن'),
 (0.550209067761898, 4904, 'علمه البيان'),
 (0.545628587404887, 5749, 'وآثر الحياة الدنيا'),
 (0.5277290642261505, 3904, 'وآتيناهما الكتاب المستبين'),
 (0.5126987975090742, 4818, 'أعنده علم الغيب فهو يرى'),
 (0.48682739213109016, 4694, 'وفي الأرض آيات للموقنين'),
 (0.4812457412481308, 6109, 'الذي علم بالقلم'),
 (0.47858272306621075, 3253, 'تلك آيات الكتاب المبين'),
 (0.47858272306621075, 2933, 'تلك آيات الكتاب المبين'),
 (0.4767161551862955, 6172, 'كلا لو تعلمون علم اليقين')]

In [94]:
get_most_similar_verses_by_query_text('المجادلة بغير علم', (model_tw, 'TWITTER'), get_verse_max_score)[:10]

[(2.571965,
  3488,
  'ألم تروا أن الله سخر لكم ما في السماوات وما في الأرض وأسبغ عليكم نعمه ظاهرة وباطنة ومن الناس من يجادل في الله بغير علم ولا هدى ولا كتاب منير'),
 (2.571965, 2602, 'ومن الناس من يجادل في الله بغير علم ولا هدى ولا كتاب منير'),
 (2.571965, 2597, 'ومن الناس من يجادل في الله بغير علم ويتبع كل شيطان مريد'),
 (2.4906023,
  896,
  'ولا تسبوا الذين يدعون من دون الله فيسبوا الله عدوا بغير علم كذلك زينا لكل أمة عملهم ثم إلى ربهم مرجعهم فينبئهم بما كانوا يعملون'),
 (2.0,
  4607,
  'هم الذين كفروا وصدوكم عن المسجد الحرام والهدي معكوفا أن يبلغ محله ولولا رجال مؤمنون ونساء مؤمنات لم تعلموهم أن تطئوهم فتصيبكم منهم معرة بغير علم ليدخل الله في رحمته من يشاء لو تزيلوا لعذبنا الذين كفروا منهم عذابا أليما'),
 (2.0,
  3474,
  'ومن الناس من يشتري لهو الحديث ليضل عن سبيل الله بغير علم ويتخذها هزوا أولئك لهم عذاب مهين'),
 (2.0,
  3437,
  'بل اتبع الذين ظلموا أهواءهم بغير علم فمن يهدي من أضل الله وما لهم من ناصرين'),
 (2.0,
  1925,
  'ليحملوا أوزارهم كاملة يوم القيامة ومن أوزار الذين يضلون

In [95]:
get_most_similar_verses_by_query_text('المجادلة بغير علم', (model_fasttext, 'FASTTEXT'), get_verse_max_score)[:10]

[(2.0,
  4607,
  'هم الذين كفروا وصدوكم عن المسجد الحرام والهدي معكوفا أن يبلغ محله ولولا رجال مؤمنون ونساء مؤمنات لم تعلموهم أن تطئوهم فتصيبكم منهم معرة بغير علم ليدخل الله في رحمته من يشاء لو تزيلوا لعذبنا الذين كفروا منهم عذابا أليما'),
 (2.0,
  3488,
  'ألم تروا أن الله سخر لكم ما في السماوات وما في الأرض وأسبغ عليكم نعمه ظاهرة وباطنة ومن الناس من يجادل في الله بغير علم ولا هدى ولا كتاب منير'),
 (2.0,
  3474,
  'ومن الناس من يشتري لهو الحديث ليضل عن سبيل الله بغير علم ويتخذها هزوا أولئك لهم عذاب مهين'),
 (2.0,
  3437,
  'بل اتبع الذين ظلموا أهواءهم بغير علم فمن يهدي من أضل الله وما لهم من ناصرين'),
 (2.0, 2602, 'ومن الناس من يجادل في الله بغير علم ولا هدى ولا كتاب منير'),
 (2.0, 2597, 'ومن الناس من يجادل في الله بغير علم ويتبع كل شيطان مريد'),
 (2.0,
  1925,
  'ليحملوا أوزارهم كاملة يوم القيامة ومن أوزار الذين يضلونهم بغير علم ألا ساء ما يزرون'),
 (2.0,
  932,
  'ومن الإبل اثنين ومن البقر اثنين قل آلذكرين حرم أم الأنثيين أما اشتملت عليه أرحام الأنثيين أم كنتم شهداء إذ وصاكم الله به

## Combining results of all models

In [100]:
def get_combined_models_results(query_text, method):
    '''
    Get the most similar verses to a query text,
    according to all models and one of the 3 maximizing methods (max, freq, avg).
    
    @param query_text: query text (one or more words)
    @type query_text: str (arabic unicode)
    @param method: the maximizing method to use
    @type method: function
    @return: most similar verses
    @rtype: list of tuples (score, verse_id, verse_text)
    '''

    verse2score = {}
    for model in [(model_wiki, 'WIKI'), (model_tw, 'TWITTER'), (model_ksucca, 'KSUCCA'), (model_fasttext, 'FASTTEXT')]:
        score2verse = get_most_similar_verses_by_query_text(query_text, model, method)
        for score, verse_id, verse in score2verse:
            if (verse_id, verse) in verse2score:
                verse2score[(verse_id, verse)] += score
            else:
                verse2score[(verse_id, verse)] = score

    best_score2verse = [(score, verse_id, verse) for (verse_id, verse), score in verse2score.items()]
    best_score2verse.sort(reverse=True)

    # Return at most 50 verses
    max_out_length = min(len(best_score2verse), 50)
    return best_score2verse[:max_out_length]

### Testing version 2

In [105]:
get_combined_models_results('التوبة', get_verse_max_score)[:20]

[(4.0, 4296, 'وهو الذي يقبل التوبة عن عباده ويعفو عن السيئات ويعلم ما تفعلون'),
 (4.0,
  1338,
  'ألم يعلموا أن الله هو يقبل التوبة عن عباده ويأخذ الصدقات وأن الله هو التواب الرحيم'),
 (4.0,
  510,
  'وليست التوبة للذين يعملون السيئات حتى إذا حضر أحدهم الموت قال إني تبت الآن ولا الذين يموتون وهم كفار أولئك أعتدنا لهم عذابا أليما'),
 (4.0,
  509,
  'إنما التوبة على الله للذين يعملون السوء بجهالة ثم يتوبون من قريب فأولئك يتوب الله عليهم وكان الله عليما حكيما'),
 (2.3758178, 5550, 'وما يذكرون إلا أن يشاء الله هو أهل التقوى وأهل المغفرة'),
 (2.3758178,
  4815,
  'الذين يجتنبون كبائر الإثم والفواحش إلا اللمم إن ربك واسع المغفرة هو أعلم بكم إذ أنشأكم من الأرض وإذ أنتم أجنة في بطون أمهاتكم فلا تزكوا أنفسكم هو أعلم بمن اتقى'),
 (2.2074943,
  4110,
  'قل يا عبادي الذين أسرفوا على أنفسهم لا تقنطوا من رحمة الله إن الله يغفر الذنوب جميعا إنه هو الغفور الرحيم'),
 (2.2074943,
  427,
  'والذين إذا فعلوا فاحشة أو ظلموا أنفسهم ذكروا الله فاستغفروا لذنوبهم ومن يغفر الذنوب إلا الله ولم يصروا على ما فعلوا

In [104]:
get_combined_models_results('الصلاة وقت الحرب', get_verse_max_score)[:20]

[(6.238496,
  569,
  'ألم تر إلى الذين قيل لهم كفوا أيديكم وأقيموا الصلاة وآتوا الزكاة فلما كتب عليهم القتال إذا فريق منهم يخشون الناس كخشية الله أو أشد خشية وقالوا ربنا لم كتبت علينا القتال لولا أخرتنا إلى أجل قريب قل متاع الدنيا قليل والآخرة خير لمن اتقى ولا تظلمون فتيلا'),
 (5.139816,
  759,
  'إنما يريد الشيطان أن يوقع بينكم العداوة والبغضاء في الخمر والميسر ويصدكم عن ذكر الله وعن الصلاة فهل أنتم منتهون'),
 (4.560791,
  680,
  'ولقد أخذ الله ميثاق بني إسرائيل وبعثنا منهم اثني عشر نقيبا وقال الله إني معكم لئن أقمتم الصلاة وآتيتم الزكاة وآمنتم برسلي وعزرتموهم وأقرضتم الله قرضا حسنا لأكفرن عنكم سيئاتكم ولأدخلنكم جنات تجري من تحتها الأنهار فمن كفر بعد ذلك منكم فقد ضل سواء السبيل'),
 (4.4125013,
  2672,
  'وجاهدوا في الله حق جهاده هو اجتباكم وما جعل عليكم في الدين من حرج ملة أبيكم إبراهيم هو سماكم المسلمين من قبل وفي هذا ليكون الرسول شهيدا عليكم وتكونوا شهداء على الناس فأقيموا الصلاة وآتوا الزكاة واعتصموا بالله هو مولاكم فنعم المولى ونعم النصير'),
 (4.3163, 3439, 'منيبين إليه واتقوه وأق

In [102]:
get_combined_models_results('بصمات الأصابع', get_verse_avg_score)[:20]

[(1.3378965047498543, 5579, 'والتفت الساق بالساق'),
 (1.1530736138423285, 5974, 'وجوه يومئذ ناعمة'),
 (0.9558047354221344, 5995, 'والشفع والوتر'),
 (0.9502903148531914, 5980, 'وأكواب موضوعة'),
 (0.7338173687458038, 5008, 'وظل ممدود'),
 (0.7199279963970184, 3833, 'بيضاء لذة للشاربين'),
 (0.6950308208664259, 5005, 'وأصحاب اليمين ما أصحاب اليمين'),
 (0.6864072009921074, 5031, 'فمالئون منها البطون'),
 (0.6493485644459724, 5019, 'وأصحاب الشمال ما أصحاب الشمال'),
 (0.6230508759617805, 6035, 'فك رقبة'),
 (0.5757891535758972, 3113, 'وزنوا بالقسطاس المستقيم'),
 (0.5681670531630516, 3905, 'وهديناهما الصراط المستقيم'),
 (0.5279583632946014, 5722, 'أإذا كنا عظاما نخرة'),
 (0.5272972472012043, 4804, 'ألكم الذكر وله الأنثى'),
 (0.5271717011928558, 4817, 'وأعطى قليلا وأكدى'),
 (0.5150741045363247, 5061, 'فلولا إذا بلغت الحلقوم'),
 (0.4957534745335579, 5553, 'أيحسب الإنسان ألن نجمع عظامه'),
 (0.4749542837962508, 1916, 'وعلامات وبالنجم هم يهتدون'),
 (0.47285299748182297, 4911, 'فيها فاكهة والنخل ذات ال

In [106]:
get_combined_models_results('الوضوء', get_verse_frequency_score)[:20]

[(31,
  674,
  'يا أيها الذين آمنوا إذا قمتم إلى الصلاة فاغسلوا وجوهكم وأيديكم إلى المرافق وامسحوا برءوسكم وأرجلكم إلى الكعبين وإن كنتم جنبا فاطهروا وإن كنتم مرضى أو على سفر أو جاء أحد منكم من الغائط أو لامستم النساء فلم تجدوا ماء فتيمموا صعيدا طيبا فامسحوا بوجوهكم وأيديكم منه ما يريد الله ليجعل عليكم من حرج ولكن يريد ليطهركم وليتم نعمته عليكم لعلكم تشكرون'),
 (31,
  202,
  'وأتموا الحج والعمرة لله فإن أحصرتم فما استيسر من الهدي ولا تحلقوا رءوسكم حتى يبلغ الهدي محله فمن كان منكم مريضا أو به أذى من رأسه ففدية من صيام أو صدقة أو نسك فإذا أمنتم فمن تمتع بالعمرة إلى الحج فما استيسر من الهدي فمن لم يجد فصيام ثلاثة أيام في الحج وسبعة إذا رجعتم تلك عشرة كاملة ذلك لمن لم يكن أهله حاضري المسجد الحرام واتقوا الله واعلموا أن الله شديد العقاب'),
 (27,
  193,
  'أحل لكم ليلة الصيام الرفث إلى نسائكم هن لباس لكم وأنتم لباس لهن علم الله أنكم كنتم تختانون أنفسكم فتاب عليكم وعفا عنكم فالآن باشروهن وابتغوا ما كتب الله لكم وكلوا واشربوا حتى يتبين لكم الخيط الأبيض من الخيط الأسود من الفجر ثم أتموا الصيام إ

## Combining(models, methods)

In [107]:
def get_all_combined(query_text):
    '''
    Get the most similar verses to a query text,
    based on combining all models with all maximizing methods.
    
    @param query_text: query text (one or more words)
    @type query_text: str (arabic unicode)
    @return: most similar verses
    @rtype: list of tuples (score, verse_id, verse_text)
    '''

    methods = [get_combined_models_results(query_text, get_verse_max_score),
               get_combined_models_results(query_text, get_verse_frequency_score),
               get_combined_models_results(query_text, get_verse_avg_score)]

    # To have a constant measurement, scores are defined based on sum of all scores
    max_score = [sum([score for score, verse_id, verse in method]) for method in methods]
    verse2score = {}
    for i in range(3):
        for score, verse_id, verse in methods[i]:
            final_score = score / max_score[i]

            # Avoid division by zero
            if final_score == 0:
                continue
            if (verse_id, verse) in verse2score:
                verse2score[(verse_id, verse)][0] += final_score
                verse2score[(verse_id, verse)][1] += 1.0
            else:
                verse2score[(verse_id, verse)] = [final_score, 1.0]

    score2verse = []
    for (verse_id, verse), [score, frequency] in verse2score.items():
        score2verse.append((score / frequency, verse_id, verse))

    score2verse.sort(reverse=True)

    # Return at most 50 verses
    max_out_length = min(len(score2verse), 50)
    return score2verse[:max_out_length]

### Testing Legendary Grand Master Version

In [108]:
get_all_combined('صلاة الجمعة')[:20]

[(0.06563245823389022,
  202,
  'وأتموا الحج والعمرة لله فإن أحصرتم فما استيسر من الهدي ولا تحلقوا رءوسكم حتى يبلغ الهدي محله فمن كان منكم مريضا أو به أذى من رأسه ففدية من صيام أو صدقة أو نسك فإذا أمنتم فمن تمتع بالعمرة إلى الحج فما استيسر من الهدي فمن لم يجد فصيام ثلاثة أيام في الحج وسبعة إذا رجعتم تلك عشرة كاملة ذلك لمن لم يكن أهله حاضري المسجد الحرام واتقوا الله واعلموا أن الله شديد العقاب'),
 (0.059665871121718374,
  193,
  'أحل لكم ليلة الصيام الرفث إلى نسائكم هن لباس لكم وأنتم لباس لهن علم الله أنكم كنتم تختانون أنفسكم فتاب عليكم وعفا عنكم فالآن باشروهن وابتغوا ما كتب الله لكم وكلوا واشربوا حتى يتبين لكم الخيط الأبيض من الخيط الأسود من الفجر ثم أتموا الصيام إلى الليل ولا تباشروهن وأنتم عاكفون في المساجد تلك حدود الله فلا تقربوها كذلك يبين الله آياته للناس لعلهم يتقون'),
 (0.04653656691140208, 5895, 'ويصلى سعيرا'),
 (0.04440220597171151,
  2848,
  'يا أيها الذين آمنوا ليستأذنكم الذين ملكت أيمانكم والذين لم يبلغوا الحلم منكم ثلاث مرات من قبل صلاة الفجر وحين تضعون ثيابكم من الظهيرة 

In [109]:
get_all_combined('التوبة')[:20]

[(0.057233597633765586,
  510,
  'وليست التوبة للذين يعملون السيئات حتى إذا حضر أحدهم الموت قال إني تبت الآن ولا الذين يموتون وهم كفار أولئك أعتدنا لهم عذابا أليما'),
 (0.04441000603079315, 5571, 'وتذرون الآخرة'),
 (0.043761638733705775,
  288,
  'يا أيها الذين آمنوا إذا تداينتم بدين إلى أجل مسمى فاكتبوه وليكتب بينكم كاتب بالعدل ولا يأب كاتب أن يكتب كما علمه الله فليكتب وليملل الذي عليه الحق وليتق الله ربه ولا يبخس منه شيئا فإن كان الذي عليه الحق سفيها أو ضعيفا أو لا يستطيع أن يمل هو فليملل وليه بالعدل واستشهدوا شهيدين من رجالكم فإن لم يكونا رجلين فرجل وامرأتان ممن ترضون من الشهداء أن تضل إحداهما فتذكر إحداهما الأخرى ولا يأب الشهداء إذا ما دعوا ولا تسأموا أن تكتبوه صغيرا أو كبيرا إلى أجله ذلكم أقسط عند الله وأقوم للشهادة وأدنى ألا ترتابوا إلا أن تكون تجارة حاضرة تديرونها بينكم فليس عليكم جناح ألا تكتبوها وأشهدوا إذا تبايعتم ولا يضار كاتب ولا شهيد وإن تفعلوا فإنه فسوق بكم واتقوا الله ويعلمكم الله والله بكل شيء عليم'),
 (0.04330871594538962,
  4296,
  'وهو الذي يقبل التوبة عن عباده ويعفو

In [110]:
get_all_combined('قول الزور')[:20]

[(0.07128460063818569,
  2624,
  'ذلك ومن يعظم حرمات الله فهو خير له عند ربه وأحلت لكم الأنعام إلا ما يتلى عليكم فاجتنبوا الرجس من الأوثان واجتنبوا قول الزور'),
 (0.04832398124870964, 2375, 'يفقهوا قولي'),
 (0.04766269477543538,
  288,
  'يا أيها الذين آمنوا إذا تداينتم بدين إلى أجل مسمى فاكتبوه وليكتب بينكم كاتب بالعدل ولا يأب كاتب أن يكتب كما علمه الله فليكتب وليملل الذي عليه الحق وليتق الله ربه ولا يبخس منه شيئا فإن كان الذي عليه الحق سفيها أو ضعيفا أو لا يستطيع أن يمل هو فليملل وليه بالعدل واستشهدوا شهيدين من رجالكم فإن لم يكونا رجلين فرجل وامرأتان ممن ترضون من الشهداء أن تضل إحداهما فتذكر إحداهما الأخرى ولا يأب الشهداء إذا ما دعوا ولا تسأموا أن تكتبوه صغيرا أو كبيرا إلى أجله ذلكم أقسط عند الله وأقوم للشهادة وأدنى ألا ترتابوا إلا أن تكون تجارة حاضرة تديرونها بينكم فليس عليكم جناح ألا تكتبوها وأشهدوا إذا تبايعتم ولا يضار كاتب ولا شهيد وإن تفعلوا فإنه فسوق بكم واتقوا الله ويعلمكم الله والله بكل شيء عليم'),
 (0.042346297641899164,
  5104,
  'بسم الله الرحمن الرحيم قد سمع الله قول التي

## Pooling

In [111]:
from pyarabic.araby import tokenize

query = "كيف نشأ الإنسان"
query_tokens = tokenize(query)
query_tokens

['كيف', 'نشأ', 'الإنسان']

In [112]:
import numpy as np

for token in query_tokens:
  print(token)
  vec1 = model_tw[clean(token, 'TWITTER')]
  vec2 = model_wiki[clean(token, 'WIKI')]
  vec3 = model_ksucca[clean(token, 'KSUCCA')]
  vec4 = model_fasttext[clean(token, 'FASTTEXT')]

  print(np.shape(vec1)) # 1-dimensional array with length 300
  print(np.shape(vec2)) # 1-dimensional array with length 300
  print(np.shape(vec3)) # 1-dimensional array with length 300
  print(np.shape(vec4)) # 1-dimensional array with length 300

كيف
(1, 300)
(1, 300)
(1, 300)
(1, 300)
نشأ
(1, 300)
(1, 300)
(1, 300)
(1, 300)
الإنسان
(1, 300)
(1, 300)
(1, 300)
(1, 300)


### MAX Pooling

In [113]:
def get_max_pooling_vec(tokens, model):
  arr = [-1e9 for idx in range(300)]
  max_pooling_vec = np.copy(np.array(arr))  # avoid read-only error

  (model_vectors, model_name) = model
  for token in tokens:
    cleaned = clean(token, model_name)
    if len(cleaned):
      cleaned = cleaned[0]
    else:
      cleaned = ''
    
    if cleaned not in model_vectors:
      continue
    vec = model_vectors[cleaned]
    for idx in range(300):
      max_pooling_vec[idx] = max(max_pooling_vec[idx], vec[idx])
  
  return max_pooling_vec

In [114]:
get_max_pooling_vec("الوفاء بالعهد", (model_fasttext, 'FASTTEXT'))

array([ 0.0628    ,  0.0737    ,  0.2638    ,  0.1122    ,  1.07319999,
        0.0854    ,  0.15899999,  0.13259999,  0.1516    ,  0.0736    ,
        0.0675    ,  0.0605    ,  0.1077    ,  0.0567    ,  0.1787    ,
        0.15710001,  0.14129999,  0.18170001,  0.0315    ,  0.1407    ,
        0.1919    ,  0.0168    ,  0.0585    ,  0.1973    ,  0.26289999,
        0.0176    ,  0.198     ,  0.23639999,  0.0711    ,  0.2059    ,
        0.2449    ,  0.1584    ,  0.0563    , -0.0796    ,  0.338     ,
        0.13150001,  0.1434    ,  0.0617    ,  0.2726    ,  0.14049999,
        0.2298    ,  0.1873    ,  0.1523    ,  0.044     ,  0.1998    ,
        0.163     ,  0.43000001,  0.1578    ,  0.0293    ,  0.1991    ,
        0.1171    ,  0.1418    ,  0.30500001,  0.27360001,  0.2423    ,
        0.4603    ,  0.1006    ,  0.32370001,  0.0212    ,  0.1114    ,
        0.2554    ,  0.112     ,  0.0382    ,  0.20280001,  0.1556    ,
        0.1092    ,  0.19840001,  0.2342    ,  0.29030001,  0.40

### AVG Pooling

In [116]:
def get_avg_pooling_vec(tokens, model):
  arr = [0 for idx in range(300)]
  avg_pooling_vec = np.copy(np.array(arr))  # avoid read-only error

  cnt = 0
  (model_vectors, model_name) = model
  
  for token in tokens:
    cleaned = clean(token, model_name)
    if len(cleaned):
      cleaned = cleaned[0]
    else:
      cleaned = ''
    
    if cleaned not in model_vectors:
      continue   
    cnt += 1
    vec = model_vectors[cleaned]
    for idx in range(300):
      avg_pooling_vec[idx] += vec[idx]

  for idx in range(300):
    avg_pooling_vec[idx] /= max(1, cnt)
  
  return avg_pooling_vec

In [119]:
get_avg_pooling_vec("التهجد وقيام الليل", (model_tw, 'TWITTER'))

array([ 0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0, -1,  0,  0,  0,  2,  0,  0,  0,  0,  0,  1,  0,
       -1, -2,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  1, -1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  2,  0,
        0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0, -1,  0,  0,
        0,  0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
       -1, -1,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0, -2,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  1, -1,  0, -1,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1, -1,  1,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1,  0,  0,  0, -1,
        0, -1,  1,  0, -1,  0,  0,  1, -1,  0,  0,  0,  0, -1, -1,  0,  0,
        0,  0,  0,  0,  0

### Testing

In [120]:
import numpy as np

def get_pooling_results(query_text, model, method):
    '''
    Get the pooling results for the given tokens,
    according to the given model and method.
    
    @param query_text: list of words
    @type query_text: list
    @param model: the model to use
    @type model: Word2Vec or KeyedVectors
    @param method: the method to use
    @type method: function
    @return: most similar verses
    @rtype: list of tuples (score, verse_id, verse)
    '''

    query_method_pooling_vec = method(query_text, model)

    verse_scores, verse_id = [], 0
    for verse in quran_clean_text:
        verse_method_pooling_vec = method(verse, model)
        cosine_similarity = np.dot(query_method_pooling_vec, verse_method_pooling_vec) / (
            np.linalg.norm(query_method_pooling_vec) * np.linalg.norm(verse_method_pooling_vec))

        # score = model_vectors.similarity(query_max_pooling_vec, verse_max_pooling_vec)
        # will fail, because we generated new vectors that doesn't belong to any model
        verse_scores.append((cosine_similarity, verse_id))
        verse_id += 1

    verse_scores.sort(reverse=True)

    # Return at most 50 verses
    max_out_length = min(len(verse_scores), 50)
    most_similar_verses = [(score, verse_id, quran_clean_text[verse_id])
                           for score, verse_id in verse_scores[:max_out_length]]

    return most_similar_verses

In [121]:
get_pooling_results("الوفاء بالعهد", (model_fasttext, 'FASTTEXT'), get_max_pooling_vec)

[(0.9781673769181193, 4833, 'وأنه أهلك عادا الأولى'),
 (0.9776652632273602,
  3166,
  'فلما جاءها نودي أن بورك من في النار ومن حولها وسبحان الله رب العالمين'),
 (0.9770342282100043, 4845, 'فاسجدوا لله واعبدوا'),
 (0.9766098275464736, 4339, 'وجعلوا له من عباده جزءا إن الإنسان لكفور مبين'),
 (0.9751549145345476, 4721, 'والسماء بنيناها بأيد وإنا لموسعون'),
 (0.9743300967305012, 3008, 'فإنهم عدو لي إلا رب العالمين'),
 (0.9729596766001537, 3969, 'والحمد لله رب العالمين'),
 (0.9720432208984134,
  2039,
  'ويدع الإنسان بالشر دعاءه بالخير وكان الإنسان عجولا'),
 (0.9720251128150306, 4793, 'فأوحى إلى عبده ما أوحى'),
 (0.9718574684028467, 4123, 'بل الله فاعبد وكن من الشاكرين'),
 (0.9708932960553968, 4236, 'ويوم يحشر أعداء الله إلى النار فهم يوزعون'),
 (0.9708535011749969, 3885, 'فأرادوا به كيدا فجعلناهم الأسفلين'),
 (0.9708413781576097,
  915,
  'لهم دار السلام عند ربهم وهو وليهم بما كانوا يعملون'),
 (0.9707222046781244,
  215,
  'فإن زللتم من بعد ما جاءتكم البينات فاعلموا أن الله عزيز حكيم'),
 (

In [122]:
get_pooling_results("التهجد وقيام الليل", (model_tw, 'TWITTER'), get_avg_pooling_vec)

[(0.9709195467257719,
  5080,
  'يولج الليل في النهار ويولج النهار في الليل وهو عليم بذات الصدور'),
 (0.960819982747272,
  3497,
  'ألم تر أن الله يولج الليل في النهار ويولج النهار في الليل وسخر الشمس والقمر كل يجري إلى أجل مسمى وأن الله بما تعملون خبير'),
 (0.960768922830523,
  1111,
  'قل يا أيها الناس إني رسول الله إليكم جميعا الذي له ملك السماوات والأرض لا إله إلا هو يحيي ويميت فآمنوا بالله ورسوله النبي الأمي الذي يؤمن بالله وكلماته واتبعوه لعلكم تهتدون'),
 (0.9579311388238991, 2281, 'وبرا بوالدتي ولم يجعلني جبارا شقيا'),
 (0.9562603624149731, 3115, 'واتقوا الذي خلقكم والجبلة الأولين'),
 (0.9562603624149731,
  2127,
  'أولم يروا أن الله الذي خلق السماوات والأرض قادر على أن يخلق مثلهم وجعل لهم أجلا لا ريب فيه فأبى الظالمون إلا كفورا'),
 (0.9558036755202326, 6000, 'التي لم يخلق مثلها في البلاد'),
 (0.9519230769230771,
  2655,
  'ذلك بأن الله يولج الليل في النهار ويولج النهار في الليل وأن الله سميع بصير'),
 (0.9518745131357371, 5633, 'لأي يوم أجلت'),
 (0.9516486024850805, 4927, 'ويبقى

# Evaluation

We manually collected random 22 different topics of 215 Verses, from Mushaf Al-Tajweed that is trusted by many Islamic authorizations, one of them is Al-Azhar Islamic Research Academy (مجمع البحوث الإسلامية بالأزهر الشريف). 

In [123]:
data = {
"الوفاء بالعهد":
['2/26', '2/27', '2/40', '2/80', '2/100', '2/177', '3/76', '3/77', '5/1', '5/7', '5/12', '6/152' , '8/42','9/4' , '9/7', '9/12', '13/20' , '13/25', '16/91', '16/92', '16/94', '16/95', '17/34', '23/8', '33/7', '33/15', '33/23', '70/32'],
"التهجد وقيام الليل":
["17/78","17/79","50/40","51/17","51/18","52/48","52/49","73/1","73/2","73/3","73/4","73/5","73/6","73/7","73/20","76/26"],
"التوبة":
['2/160' ,'3/86', '3/87', '3/88', '3/89', '3/90' , '3/135' , '3/136', '4/17' , '4/18', '4/26', '4/110', '5/39', '7/153' , '9/104', '9/112', '11/3' , '11/4', '11/5', '17/25', '19/60','25/70', '25/71', '39/53', '39/54', '42/25','66/8','85/10'],
"الصلاة وقت الحرب":
['4/101' , '4/102' , '4/103'],
"صفات المصلين":
["23/2","23/9","70/22","70/23","70/34","70/35"],
"أسماء الله الحسنى":
["7/180", "17/110", "20/8", "59/24"],
"المجادلة بغير علم":
["22/3", "22/8", "31/20"],
"علوم البلاغة":
['6/112' , '55/1' , '55/2', '55/3', '55/4'],
"قول الزور":
['22/30', '25/72'],
"صلاة الجمعة":
["62/9"],
"غزوة الحديبية وبيعة الرضوان":
['48/1', '48/2', '48/3', '48/4', '48/5', '48/6', '48/7', '48/8', '48/9', '48/10', '48/11', '48/12', '48/13', '48/14', '48/15', '48/16',
 '48/17', '48/18', '48/19', '48/20', '48/21', '48/22', '48/23', '48/24', '48/25', '48/26', '48/27'],
"الابتلاء والفتن":
['2/155', '2/214' , '3/152', '3/154' , '3/179' , '3/186' , '5/51', '6/165', '11/7', '21/35' , '2/29' , '47/31', '67/2'],
"سلامة القلب":
['6/127' , '8/61' , '10/9' , '10/10', '13/24' , '19/62', '21/102' , '25/63', '33/44', '39/73', '56/26'],
"بصمات الاصابع":
['75/3', '75/4'],
"ذم الجهل والجاهلين":
["7/199", "11/46", "16/119", "25/63"],
"حول ما يدعى بالتطور":
["2/29", "2/30", "2/259", "6/38", "7/11", "22/7", "71/14", "75/37", "75/38", "75/39", "75/40", "76/6", "86/5", "86/6", "86/7", "86/8"],
"آداب المجلس":
["58/9", "58/11", "58/12"],
"صلاة الخوف":
["4/101","4/102"],
"الاستذان في أوقات الخلوة":
["24/58","24/59","24/60"],
"القضاء والقدر":
['3/145' , '3/154', '6/35', '6/57', '6/96', '7/34', '9/51', '10/3', '10/49', '10/99', '10/100' , '11/6', '13/39', '15/4', '15/5', '15/21', '17/58','23/43', '25/2','27/74','27/75', '34/3', '35/11', '44/4', '54/51', '54/53', '57/22', '59/3','64/11', '65/3', '65/12', '71/4', '72/25', '72/28'],
"حد المحاربة":
['5/33'],
"الوضوء":
['4/43', '5/6', '5/7']
}

In [126]:
print('Number of topics:', len(data))
cnt = 0
for key, value in data.items():
  cnt += len(value)
print('Number of Verses:', cnt)

Number of topics: 22
Number of Verses: 215


In [127]:
def compare_us(output, expected):
  output = set(output)
  passed = []
  for item in expected:
    if item in output:
      passed.append(item)
  return passed

In [128]:
def fetch_api(surah_verse):
  test_results = []
  for verse in surah_verse:
    url = f"https://ahr9n-quran-api.herokuapp.com/api/get-verse/{verse}"
    headers = {'content-type': 'application/json'}
    results = requests.get(url, headers=headers)
    results = results.json()
    test_results.append(int(results['data']['numberInQuran']))
  return test_results

## Testing all models with all methods by `get_most_similar_verses_by_query_text`

In [129]:
import requests  
import json

def test_get_most_similar_verses_by_query_text(test_data):
  for query_text, surah_verse in test_data.items():
    print(query_text, ':', sep='')
    for model in [(model_wiki, 'WIKI'), (model_tw, 'TWITTER'), (model_ksucca, "KSUCCA"), (model_fasttext, "FASTTEXT")]:
      print(model[1], ':', sep='')
      for method_name, method in {"MAX": get_verse_max_score, "FREQ": get_verse_frequency_score, "AVG": get_verse_avg_score}.items():
        model_results = get_most_similar_verses_by_query_text(query_text, model, method)
        model_results = [int(item[1])+1 for item in model_results]
        test_results = fetch_api(surah_verse)
        
        passed = compare_us(model_results, test_results)
        print(method_name, f"{len(passed)}/{len(test_results)}", sep='-')

In [ ]:
test_get_most_similar_verses_by_query_text(data)

الوفاء بالعهد:
WIKI:
MAX-1/28
FREQ-2/28
AVG-0/28
TWITTER:
MAX-7/28
FREQ-1/28
AVG-0/28
KSUCCA:
MAX-17/28
FREQ-15/28
AVG-5/28
FASTTEXT:
MAX-23/28
FREQ-5/28
AVG-9/28
التهجد وقيام الليل:
WIKI:
MAX-11/16
FREQ-1/16
AVG-3/16
TWITTER:
MAX-10/16
FREQ-1/16
AVG-7/16
KSUCCA:
MAX-10/16
FREQ-10/16
AVG-9/16
FASTTEXT:
MAX-10/16
FREQ-5/16
AVG-9/16
التوبة:
WIKI:
MAX-6/28
FREQ-1/28
AVG-1/28
TWITTER:
MAX-7/28
FREQ-6/28
AVG-4/28
KSUCCA:
MAX-16/28
FREQ-15/28
AVG-13/28
FASTTEXT:
MAX-15/28
FREQ-3/28
AVG-6/28
الصلاة وقت الحرب:
WIKI:
MAX-2/3
FREQ-1/3
AVG-1/3
TWITTER:
MAX-2/3
FREQ-1/3
AVG-1/3
KSUCCA:
MAX-3/3
FREQ-0/3
AVG-1/3
FASTTEXT:
MAX-3/3
FREQ-0/3
AVG-1/3
صفات المصلين:
WIKI:
MAX-4/6
FREQ-0/6
AVG-1/6
TWITTER:
MAX-4/6
FREQ-0/6
AVG-4/6
KSUCCA:
MAX-4/6
FREQ-0/6
AVG-5/6
FASTTEXT:
MAX-1/6
FREQ-0/6
AVG-3/6
أسماء الله الحسنى:
WIKI:
MAX-0/4
FREQ-0/4
AVG-1/4
TWITTER:
MAX-4/4
FREQ-0/4
AVG-1/4
KSUCCA:
MAX-1/4
FREQ-1/4
AVG-1/4
FASTTEXT:
MAX-4/4
FREQ-1/4
AVG-3/4
المجادلة بغير علم:
WIKI:
MAX-3/3
FREQ-0/3
AVG-0/3
TWITTER:
M

## Testing combined models for each method by `get_combined_models_results`

In [130]:
def test_combined_models_results(test_data):
  for query_text, surah_verse in test_data.items():
    print(query_text, ':', sep='')
    for method_name, method in {"MAX": get_verse_max_score, "FREQ": get_verse_frequency_score, "AVG": get_verse_avg_score}.items():
      model_results = get_combined_models_results(query_text, method)
      model_results = [int(item[1])+1 for item in model_results]
      test_results = fetch_api(surah_verse)
      
      passed = compare_us(model_results, test_results)
      print('COMBINED_MODELS', method_name, f"{len(passed)}/{len(test_results)}", sep='-')

In [ ]:
test_combined_models_results(data)

الوفاء بالعهد:
COMBINED_MODELS-MAX-5/28
COMBINED_MODELS-FREQ-0/28
COMBINED_MODELS-AVG-2/28
التهجد وقيام الليل:
COMBINED_MODELS-MAX-10/16
COMBINED_MODELS-FREQ-0/16
COMBINED_MODELS-AVG-5/16
التوبة:
COMBINED_MODELS-MAX-12/28
COMBINED_MODELS-FREQ-2/28
COMBINED_MODELS-AVG-2/28
الصلاة وقت الحرب:
COMBINED_MODELS-MAX-3/3
COMBINED_MODELS-FREQ-0/3
COMBINED_MODELS-AVG-1/3
صفات المصلين:
COMBINED_MODELS-MAX-4/6
COMBINED_MODELS-FREQ-0/6
COMBINED_MODELS-AVG-2/6
أسماء الله الحسنى:
COMBINED_MODELS-MAX-0/4
COMBINED_MODELS-FREQ-0/4
COMBINED_MODELS-AVG-0/4
المجادلة بغير علم:
COMBINED_MODELS-MAX-3/3
COMBINED_MODELS-FREQ-0/3
COMBINED_MODELS-AVG-0/3
علوم البلاغة:
COMBINED_MODELS-MAX-0/5
COMBINED_MODELS-FREQ-0/5
COMBINED_MODELS-AVG-2/5
قول الزور:
COMBINED_MODELS-MAX-2/2
COMBINED_MODELS-FREQ-0/2
COMBINED_MODELS-AVG-0/2
صلاة الجمعة:
COMBINED_MODELS-MAX-1/1
COMBINED_MODELS-FREQ-0/1
COMBINED_MODELS-AVG-0/1
غزوة الحديبية وبيعة الرضوان:
COMBINED_MODELS-MAX-2/27
COMBINED_MODELS-FREQ-2/27
COMBINED_MODELS-AVG-0/27
الا

## Testing all combined models and methods by `get_all_combined`

In [131]:
def test_all_combined(test_data):
  for query_text, surah_verse in test_data.items():
    print(query_text, ':', sep='')

    model_results = get_all_combined(query_text)
    model_results = [int(item[1])+1 for item in model_results]
    test_results = fetch_api(surah_verse)
    
    passed = compare_us(model_results, test_results)
    print('COMBINED_ALL', f"{len(passed)}/{len(test_results)}", sep='-')

In [ ]:
test_all_combined(data)

الوفاء بالعهد:
COMBINED_ALL-1/28
التهجد وقيام الليل:
COMBINED_ALL-8/16
التوبة:
COMBINED_ALL-3/28
الصلاة وقت الحرب:
COMBINED_ALL-2/3
صفات المصلين:
COMBINED_ALL-1/6
أسماء الله الحسنى:
COMBINED_ALL-0/4
المجادلة بغير علم:
COMBINED_ALL-3/3
علوم البلاغة:
COMBINED_ALL-2/5
قول الزور:
COMBINED_ALL-2/2
صلاة الجمعة:
COMBINED_ALL-1/1
غزوة الحديبية وبيعة الرضوان:
COMBINED_ALL-0/27
الابتلاء والفتن:
COMBINED_ALL-1/13
سلامة القلب:
COMBINED_ALL-0/11
بصمات الاصابع:
COMBINED_ALL-1/2
ذم الجهل والجاهلين:
COMBINED_ALL-2/4
حول ما يدعى بالتطور:
COMBINED_ALL-0/16
آداب المجلس:
COMBINED_ALL-0/3
صلاة الخوف:
COMBINED_ALL-0/2
الاستذان في أوقات الخلوة:
COMBINED_ALL-0/3
القضاء والقدر:
COMBINED_ALL-0/34
حد المحاربة:
COMBINED_ALL-0/1
الوضوء:
COMBINED_ALL-1/3


## Test Pooling

In [ ]:
def test_pooling(test_data):
  for query_text, surah_verse in test_data.items():
    print(query_text, ':', sep='')
    for model in [(model_wiki, 'WIKI'), (model_tw, 'TWITTER'), (model_ksucca, "KSUCCA"), (model_fasttext, "FASTTEXT")]:
      print(model[1], ':', sep='')
      for method_name, method in {'MAX': get_max_pooling_vec, 'AVG': get_avg_pooling_vec}.items():
        model_results = get_pooling_results(query_text, model, method)
        model_results = [int(item[1])+1 for item in model_results]
        test_results = fetch_api(surah_verse)
        
        passed = compare_us(model_results, test_results)
        print(f'{method_name}_POOLING', f"{len(passed)}/{len(test_results)}", sep='-')

# TODOs

* Use tranformers (even if we don't have much knowledge with) to only compare results of this work with results of considerable transformers real sentence embedding, so our target would be to show better results with only word2vec by those maximizing techniques.

  * Done in [Transformers Notebook](https://colab.research.google.com/drive/12VQljEPWzdfmUV7AxXjtHelb8lBxzRQT).